# 패키지 불러오기

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


n_splits = 5

# 모델 정의
 1. Lightgbm
 2. Catboost


In [2]:
def Xy_split(dataset):
    
    dataset_X = dataset.drop('연봉', axis=1)
    dataset_y = dataset['연봉']
    
    return dataset_X, dataset_y

In [3]:
def lgbm_model(data_x, data_y):

    # lgbm 모델링
    lgb_params = {"objective" : "rmse",
                 "verbosity" : -1}

    lgb_models={}
    step = 1

    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        print(f'\n\n ============================ {step} ============================')    

        X_train = data_x.iloc[tr_idx, :].values
        y_train = data_y[tr_idx].values

        X_valid = data_x.iloc[val_idx, :].values
        y_valid = data_y[val_idx].values

        lgb_dtrain = lgb.Dataset(data = X_train, label = y_train) 
        lgb_dvalid = lgb.Dataset(data = X_valid, label = y_valid) 

        lgb_model = lgb.train(lgb_params, lgb_dtrain, 20000, valid_sets=[lgb_dvalid], early_stopping_rounds=100, verbose_eval=1000)
        lgb_models[step] = lgb_model

        step += 1
        
    # lgbm 성능종합
    performance = [lgb_models[step].best_score['valid_0']['rmse'] for step in range(1,6)]
    cv_performance = round(np.mean(performance),1)

    return cv_performance

In [4]:
def cb_model(data_x, data_y):

    # cb 모델링
    cat_cols = []

    cb_models={}
    step = 1

    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        X_train = data_x.iloc[tr_idx, :]
        y_train = data_y[tr_idx]

        X_valid = data_x.iloc[val_idx, :]
        y_valid = data_y[val_idx]

        cb_dtrain = Pool(data=X_train, label=y_train, cat_features=cat_cols)
        cb_dvalid = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

        cb_model = CatBoostRegressor(iterations=20000, eval_metric='RMSE', loss_function='RMSE', verbose = 0)

        cb_model.fit(cb_dtrain, eval_set=cb_dvalid, early_stopping_rounds=100, verbose_eval=1000, use_best_model=True)
        cb_models[step] = cb_model

        step += 1
        
    # cb 성능종합
    performance = [cb_models[step].best_score_['validation']['RMSE'] for step in range(1,6)]
    cv_performance = round(np.mean(performance),1)

    return cv_performance

# train, test분할

In [5]:
hitter_simple_fa = pd.read_csv('../선수데이터(전처리완료)/모델링용ver7/단순가중/타자_단순가중_fa_train.csv').drop(['ID','선수명'], axis=1)
hitter_simple_nonfa = pd.read_csv('../선수데이터(전처리완료)/모델링용ver7/단순가중/타자_단순가중_nonfa_train.csv').drop(['ID','선수명'], axis=1)
pitcher_simple_fa = pd.read_csv('../선수데이터(전처리완료)/모델링용ver7/단순가중/투수_단순가중_fa_train.csv').drop(['ID','선수명'], axis=1)
pitcher_simple_nonfa = pd.read_csv('../선수데이터(전처리완료)/모델링용ver7/단순가중/투수_단순가중_nonfa_train.csv').drop(['ID','선수명'], axis=1)

hitter_time_weighted_fa = pd.read_csv('../선수데이터(전처리완료)/모델링용ver7/시간가중/타자_시간가중_fa_train.csv').drop(['ID','선수명'], axis=1)
hitter_time_weighted_nonfa = pd.read_csv('../선수데이터(전처리완료)/모델링용ver7/시간가중/타자_시간가중_nonfa_train.csv').drop(['ID','선수명'], axis=1)
pitcher_time_weighted_fa = pd.read_csv('../선수데이터(전처리완료)/모델링용ver7/시간가중/투수_시간가중_fa_train.csv').drop(['ID','선수명'], axis=1)
pitcher_time_weighted_nonfa = pd.read_csv('../선수데이터(전처리완료)/모델링용ver7/시간가중/투수_시간가중_nonfa_train.csv').drop(['ID','선수명'], axis=1)

hitter_play_weighted_fa = pd.read_csv('../선수데이터(전처리완료)/모델링용ver7/경기가중/타자_경기가중_fa_train.csv').drop(['ID','선수명'], axis=1)
hitter_play_weighted_nonfa = pd.read_csv('../선수데이터(전처리완료)/모델링용ver7/경기가중/타자_경기가중_nonfa_train.csv').drop(['ID','선수명'], axis=1)
pitcher_play_weighted_fa = pd.read_csv('../선수데이터(전처리완료)/모델링용ver7/경기가중/투수_경기가중_fa_train.csv').drop(['ID','선수명'], axis=1)
pitcher_play_weighted_nonfa = pd.read_csv('../선수데이터(전처리완료)/모델링용ver7/경기가중/투수_경기가중_nonfa_train.csv').drop(['ID','선수명'], axis=1)

hitter_time_and_play_weighted_fa = pd.read_csv('../선수데이터(전처리완료)/모델링용ver7/시간경기가중/타자_시간경기가중_fa_train.csv').drop(['ID','선수명'], axis=1)
hitter_time_and_play_weighted_nonfa = pd.read_csv('../선수데이터(전처리완료)/모델링용ver7/시간경기가중/타자_시간경기가중_nonfa_train.csv').drop(['ID','선수명'], axis=1)
pitcher_time_and_play_weighted_fa = pd.read_csv('../선수데이터(전처리완료)/모델링용ver7/시간경기가중/투수_시간경기가중_fa_train.csv').drop(['ID','선수명'], axis=1)
pitcher_time_and_play_weighted_nonfa = pd.read_csv('../선수데이터(전처리완료)/모델링용ver7/시간경기가중/투수_시간경기가중_nonfa_train.csv').drop(['ID','선수명'], axis=1)



hitter_datas = [hitter_simple_fa, hitter_time_weighted_fa, hitter_play_weighted_fa, hitter_time_and_play_weighted_fa,
                hitter_simple_nonfa, hitter_time_weighted_nonfa, hitter_play_weighted_nonfa, hitter_time_and_play_weighted_nonfa]

pitcher_datas = [pitcher_simple_fa, pitcher_time_weighted_fa, pitcher_play_weighted_fa, pitcher_time_and_play_weighted_fa,
                pitcher_simple_nonfa, pitcher_time_weighted_nonfa, pitcher_play_weighted_nonfa, pitcher_time_and_play_weighted_nonfa]

colnames = ['FA단순가중', 'FA시간가중', 'FA경기가중', 'FA시간경기가중', 'NONFA단순가중', 'NONFA시간가중', 'NONFA경기가중', 'NONFA시간경기가중']

# 1. Baseline

#### (1) 타자

In [6]:
hitter_baseline = pd.DataFrame()

for hitter_data, colname in zip(hitter_datas, colnames):
    
    X, y = Xy_split(hitter_data)
    tmp = pd.DataFrame([lgbm_model(X, y), cb_model(X, y)], columns=[colname])
    hitter_baseline = pd.concat([hitter_baseline, tmp], axis=1)
    
hitter_baseline



 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[53]	valid_0's rmse: 18961.2


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[143]	valid_0's rmse: 23061


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[97]	valid_0's rmse: 14447.5


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[313]	valid_0's rmse: 16755.2


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[90]	valid_0's rmse: 21517
Learning rate set to 0.007132
0:	learn: 50873.9113456	tes

1000:	learn: 10027.4030776	test: 21920.7659426	best: 21920.7659426 (1000)	total: 13s	remaining: 4m 7s
2000:	learn: 6474.7266390	test: 20766.2378211	best: 20766.2378211 (2000)	total: 26.2s	remaining: 3m 55s
3000:	learn: 4354.3894734	test: 20421.7008714	best: 20421.7008714 (3000)	total: 40s	remaining: 3m 46s
4000:	learn: 3017.9000046	test: 20240.7040213	best: 20240.7040213 (4000)	total: 53.1s	remaining: 3m 32s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 20164.11258
bestIteration = 4720

Shrink model to first 4721 iterations.
Learning rate set to 0.007132
0:	learn: 51098.9979642	test: 48307.3680418	best: 48307.3680418 (0)	total: 34.4ms	remaining: 11m 28s
1000:	learn: 9912.1552398	test: 15193.4890664	best: 15193.3438345 (999)	total: 13.7s	remaining: 4m 19s
2000:	learn: 6343.6593171	test: 14402.8433705	best: 14402.0706304 (1995)	total: 27.1s	remaining: 4m 4s
3000:	learn: 4236.8271563	test: 14092.1683353	best: 14091.7316980 (2999)	total: 40.2s	remaining: 3m 47s
4000:	l

5000:	learn: 1895.4536369	test: 14794.0586298	best: 14793.0696835 (4979)	total: 1m 3s	remaining: 3m 10s
6000:	learn: 1335.8294120	test: 14752.1798067	best: 14751.4691163 (5986)	total: 1m 15s	remaining: 2m 56s
7000:	learn: 953.5733066	test: 14732.4526327	best: 14732.4137752 (6998)	total: 1m 28s	remaining: 2m 44s
8000:	learn: 691.5173339	test: 14716.8969120	best: 14716.8969120 (8000)	total: 1m 43s	remaining: 2m 34s
9000:	learn: 499.4358239	test: 14705.2661927	best: 14705.2352948 (8997)	total: 1m 56s	remaining: 2m 22s
10000:	learn: 365.8806948	test: 14697.9425813	best: 14697.9024229 (9996)	total: 2m 9s	remaining: 2m 8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 14695.38871
bestIteration = 10416

Shrink model to first 10417 iterations.
Learning rate set to 0.007133
0:	learn: 50790.4939739	test: 49688.1009491	best: 49688.1009491 (0)	total: 12.7ms	remaining: 4m 14s
1000:	learn: 10117.9677236	test: 18892.3857817	best: 18891.9215005 (999)	total: 13.3s	remaining: 4m 12s


[12000]	valid_0's rmse: 16146.4
[13000]	valid_0's rmse: 16146.4
[14000]	valid_0's rmse: 16146.4
[15000]	valid_0's rmse: 16146.4
[16000]	valid_0's rmse: 16146.4
[17000]	valid_0's rmse: 16146.4
[18000]	valid_0's rmse: 16146.4
[19000]	valid_0's rmse: 16146.4
[20000]	valid_0's rmse: 16146.4
Did not meet early stopping. Best iteration is:
[19981]	valid_0's rmse: 16146.4


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[9]	valid_0's rmse: 3723.31
Learning rate set to 0.007619
0:	learn: 14128.5996119	test: 16330.4591752	best: 16330.4591752 (0)	total: 19.5ms	remaining: 6m 30s
1000:	learn: 2450.6403427	test: 7380.3928699	best: 7380.3597879 (999)	total: 11.8s	remaining: 3m 44s
2000:	learn: 1444.0767522	test: 6385.7847732	best: 6385.7847732 (2000)	total: 23.7s	remaining: 3m 33s
3000:	learn: 957.9088695	test: 6304.9708217	best: 6304.9708217 (3000)	total: 35.2s	remaining: 3m 19s
4000:	lear

4000:	learn: 555.5797490	test: 2377.2630836	best: 2377.2630836 (4000)	total: 51.4s	remaining: 3m 25s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 2374.939893
bestIteration = 4426

Shrink model to first 4427 iterations.


 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[97]	valid_0's rmse: 10718.5


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[16]	valid_0's rmse: 7450.72


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[11]	valid_0's rmse: 3248.06


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's rmse: 16482.7
[2000]	valid_0's rmse: 16349.1
[3000

Stopped by overfitting detector  (100 iterations wait)

bestTest = 14425.41562
bestIteration = 3759

Shrink model to first 3760 iterations.
Learning rate set to 0.00762
0:	learn: 15933.6951045	test: 6840.9098132	best: 6840.9098132 (0)	total: 23.8ms	remaining: 7m 55s
1000:	learn: 1977.9637790	test: 2656.1897055	best: 2656.1599300 (999)	total: 12.4s	remaining: 3m 55s
2000:	learn: 1113.6892215	test: 2491.7695187	best: 2491.7695187 (2000)	total: 25s	remaining: 3m 44s
3000:	learn: 745.2339927	test: 2430.1581963	best: 2430.1181168 (2994)	total: 38.2s	remaining: 3m 36s
4000:	learn: 526.6272893	test: 2412.3406456	best: 2412.2937725 (3997)	total: 51.1s	remaining: 3m 24s
5000:	learn: 388.4304548	test: 2407.2753723	best: 2407.2430007 (4976)	total: 1m 3s	remaining: 3m 10s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 2406.577937
bestIteration = 5098

Shrink model to first 5099 iterations.


,FA단순가중,FA시간가중,FA경기가중,FA시간경기가중,NONFA단순가중,NONFA시간가중,NONFA경기가중,NONFA시간경기가중
0,18948.4,18505.6,18931.6,18531.1,8297.4,8280.9,8301.2,8406.8
1,16311.2,16244.4,16035.7,16205.4,6449.4,6661.8,6563.7,6684.4


#### (2) 투수

In [7]:
pitcher_baseline = pd.DataFrame()

for pitcher_data, colname in zip(pitcher_datas, colnames):
    
    X, y = Xy_split(pitcher_data)
    tmp = pd.DataFrame([lgbm_model(X, y), cb_model(X, y)], columns=[colname])
    pitcher_baseline = pd.concat([pitcher_baseline, tmp], axis=1)
    
pitcher_baseline



 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[373]	valid_0's rmse: 11319.4


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[350]	valid_0's rmse: 17271.1


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[229]	valid_0's rmse: 18278.9


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[780]	valid_0's rmse: 12681.9


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[61]	valid_0's rmse: 12866.2
Learning rate set to 0.006285
0:	learn: 47195.99552

14000:	learn: 24.4105678	test: 14709.7280787	best: 14709.7265417 (13948)	total: 2m 39s	remaining: 1m 8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 14709.68858
bestIteration = 14079

Shrink model to first 14080 iterations.
Learning rate set to 0.006285
0:	learn: 46147.3612521	test: 47021.3609243	best: 47021.3609243 (0)	total: 10.8ms	remaining: 3m 36s
1000:	learn: 6962.6765658	test: 18047.2566295	best: 18047.2566295 (1000)	total: 12.1s	remaining: 3m 49s
2000:	learn: 3531.6620015	test: 16925.7308566	best: 16925.5847577 (1999)	total: 23.5s	remaining: 3m 30s
3000:	learn: 2061.9799279	test: 16745.9775026	best: 16745.7711644 (2997)	total: 34.4s	remaining: 3m 14s
4000:	learn: 1239.3291336	test: 16705.5412781	best: 16705.5412781 (4000)	total: 45.2s	remaining: 3m
5000:	learn: 771.8298075	test: 16682.4011060	best: 16682.4011060 (5000)	total: 56.3s	remaining: 2m 48s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 16678.72283
bestIteration = 5121

Shrink m

4000:	learn: 1345.3199776	test: 14034.8238424	best: 14034.8238424 (4000)	total: 47.4s	remaining: 3m 9s
5000:	learn: 892.8054278	test: 13996.3826284	best: 13996.3779589 (4999)	total: 58.5s	remaining: 2m 55s
6000:	learn: 580.8646665	test: 13973.8526288	best: 13973.8018623 (5995)	total: 1m 9s	remaining: 2m 43s
7000:	learn: 379.0971360	test: 13961.4755002	best: 13961.4755002 (7000)	total: 1m 21s	remaining: 2m 31s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 13955.47621
bestIteration = 7573

Shrink model to first 7574 iterations.
Learning rate set to 0.006287
0:	learn: 45907.7044643	test: 47753.8582329	best: 47753.8582329 (0)	total: 11.5ms	remaining: 3m 49s
1000:	learn: 6699.4381888	test: 14080.5792582	best: 14080.5792582 (1000)	total: 12.1s	remaining: 3m 49s
2000:	learn: 3414.5288332	test: 12988.8585201	best: 12988.8585201 (2000)	total: 23.7s	remaining: 3m 33s
3000:	learn: 1948.9735422	test: 12811.8594570	best: 12811.8594570 (3000)	total: 35.1s	remaining: 3m 18s
4000:

1000:	learn: 2775.2887444	test: 5357.1111597	best: 5356.6817927 (994)	total: 10.2s	remaining: 3m 13s
2000:	learn: 1694.9985691	test: 5144.2161714	best: 5143.7426988 (1987)	total: 20.5s	remaining: 3m 4s
3000:	learn: 1160.9892215	test: 5090.7262044	best: 5090.6485806 (2999)	total: 30.8s	remaining: 2m 54s
4000:	learn: 823.2108232	test: 5071.0242792	best: 5070.9528933 (3999)	total: 41.1s	remaining: 2m 44s
5000:	learn: 604.6714439	test: 5049.9767187	best: 5049.9767187 (5000)	total: 51.3s	remaining: 2m 33s
6000:	learn: 450.2445010	test: 5040.6708197	best: 5040.4457271 (5984)	total: 1m 1s	remaining: 2m 24s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 5038.16478
bestIteration = 6347

Shrink model to first 6348 iterations.
Learning rate set to 0.007555
0:	learn: 9563.5778167	test: 16958.4778445	best: 16958.4778445 (0)	total: 23.3ms	remaining: 7m 45s
1000:	learn: 2675.9971577	test: 12913.0936199	best: 12913.0936199 (1000)	total: 10.5s	remaining: 3m 19s
2000:	learn: 1672.153

Stopped by overfitting detector  (100 iterations wait)

bestTest = 3321.325542
bestIteration = 721

Shrink model to first 722 iterations.
Learning rate set to 0.007556
0:	learn: 11761.6473804	test: 9966.4785534	best: 9966.4785534 (0)	total: 12.6ms	remaining: 4m 12s
1000:	learn: 2791.8571466	test: 4792.6184041	best: 4792.6184041 (1000)	total: 12.2s	remaining: 3m 51s
2000:	learn: 1734.6520442	test: 4605.8770511	best: 4605.8770511 (2000)	total: 24.8s	remaining: 3m 42s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 4579.282956
bestIteration = 2697

Shrink model to first 2698 iterations.


 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[824]	valid_0's rmse: 4637.85


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[206]	valid_0's rmse: 12610.1


 =======

,FA단순가중,FA시간가중,FA경기가중,FA시간경기가중,NONFA단순가중,NONFA시간가중,NONFA경기가중,NONFA시간경기가중
0,14483.5,14311.2,14620.7,14796.0,6451.4,6342.4,6438.4,6472.0
1,14259.1,13806.6,13892.7,13684.1,6710.5,6894.9,6780.1,6810.2


# 2. 표준화 (Z-score)

In [8]:
from sklearn.preprocessing import StandardScaler

#### (1) 타자

In [9]:
hitter_zscore = pd.DataFrame()

for hitter_data, colname in zip(hitter_datas, colnames):
    
    X, y = Xy_split(hitter_data)
    
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X)
    
    tmp = pd.DataFrame([lgbm_model(X, y), cb_model(X, y)], columns=[colname])
    hitter_zscore = pd.concat([hitter_zscore, tmp], axis=1)
    
hitter_zscore



 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[51]	valid_0's rmse: 19435


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[125]	valid_0's rmse: 23044.9


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[77]	valid_0's rmse: 14303.1


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[435]	valid_0's rmse: 16761.5


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[57]	valid_0's rmse: 20616.5
Learning rate set to 0.007132
0:	learn: 50873.9113456	t

3000:	learn: 4354.3894734	test: 20429.7199938	best: 20429.7199938 (3000)	total: 36.3s	remaining: 3m 25s
4000:	learn: 3017.9000046	test: 20249.2698367	best: 20249.2698367 (4000)	total: 49.1s	remaining: 3m 16s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 20172.94065
bestIteration = 4720

Shrink model to first 4721 iterations.
Learning rate set to 0.007132
0:	learn: 51098.9979642	test: 48307.3680418	best: 48307.3680418 (0)	total: 13.1ms	remaining: 4m 20s
1000:	learn: 9912.2063276	test: 15193.1549708	best: 15193.0097441 (999)	total: 12.5s	remaining: 3m 57s
2000:	learn: 6349.6473548	test: 14419.7275840	best: 14419.4731105 (1999)	total: 24.4s	remaining: 3m 39s
3000:	learn: 4232.3375175	test: 14073.2305082	best: 14073.2305082 (3000)	total: 35.9s	remaining: 3m 23s
4000:	learn: 2861.2133828	test: 13948.0301133	best: 13947.7762368 (3991)	total: 47.9s	remaining: 3m 11s
5000:	learn: 1959.6966065	test: 13888.9312357	best: 13888.9312357 (5000)	total: 1m	remaining: 3m 1s
6000:	l

1000:	learn: 10133.4047412	test: 18713.8344357	best: 18713.8344357 (1000)	total: 12.3s	remaining: 3m 54s
2000:	learn: 6350.6313780	test: 17302.2381141	best: 17301.2881841 (1995)	total: 24.3s	remaining: 3m 38s
3000:	learn: 3864.8385236	test: 16781.9785986	best: 16781.9785986 (3000)	total: 37s	remaining: 3m 29s
4000:	learn: 2573.5824158	test: 16594.5807035	best: 16594.2996380 (3998)	total: 49.6s	remaining: 3m 18s
5000:	learn: 1768.2317388	test: 16510.1215673	best: 16509.8094873 (4968)	total: 1m 1s	remaining: 3m 5s
6000:	learn: 1226.6007478	test: 16468.3799713	best: 16467.5018768 (5970)	total: 1m 13s	remaining: 2m 51s
7000:	learn: 910.9433793	test: 16445.9174211	best: 16445.8050483 (6977)	total: 1m 25s	remaining: 2m 39s
8000:	learn: 664.5878232	test: 16433.8261410	best: 16433.8146051 (7999)	total: 1m 38s	remaining: 2m 28s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 16427.87247
bestIteration = 8580

Shrink model to first 8581 iterations.


 ==========================

Stopped by overfitting detector  (100 iterations wait)

bestTest = 6295.529513
bestIteration = 4468

Shrink model to first 4469 iterations.
Learning rate set to 0.007619
0:	learn: 14943.3670156	test: 13068.3398528	best: 13068.3398528 (0)	total: 19.1ms	remaining: 6m 21s
1000:	learn: 2001.2426209	test: 6829.0407337	best: 6829.0407337 (1000)	total: 12.4s	remaining: 3m 55s
2000:	learn: 1049.8433843	test: 6539.7426770	best: 6539.6997454 (1999)	total: 25.1s	remaining: 3m 45s
3000:	learn: 720.8368553	test: 6473.3956658	best: 6473.3956658 (3000)	total: 37.7s	remaining: 3m 33s
4000:	learn: 516.0023869	test: 6453.8185220	best: 6453.6965132 (3990)	total: 49.8s	remaining: 3m 19s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 6453.696513
bestIteration = 3990

Shrink model to first 3991 iterations.
Learning rate set to 0.007619
0:	learn: 15879.3134909	test: 7386.2802567	best: 7386.2802567 (0)	total: 20ms	remaining: 6m 39s
1000:	learn: 1983.6254718	test: 3383.9602905	best: 3383.96

[3000]	valid_0's rmse: 16195.9
[4000]	valid_0's rmse: 16195.8
Early stopping, best iteration is:
[4729]	valid_0's rmse: 16195.8


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[10]	valid_0's rmse: 3745.04
Learning rate set to 0.007619
0:	learn: 14139.0759123	test: 16294.6702244	best: 16294.6702244 (0)	total: 13.2ms	remaining: 4m 23s
1000:	learn: 2427.2539422	test: 8133.0760925	best: 8133.0760925 (1000)	total: 12.6s	remaining: 3m 59s
2000:	learn: 1453.5843656	test: 7086.4231810	best: 7086.3030483 (1998)	total: 25.4s	remaining: 3m 48s
3000:	learn: 928.2918191	test: 6975.5285643	best: 6975.5285643 (3000)	total: 37.7s	remaining: 3m 33s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 6953.760988
bestIteration = 3748

Shrink model to first 3749 iterations.
Learning rate set to 0.007619
0:	learn: 14952.0687547	test: 13036.8478873	best: 13036.8478873 (0)	total: 11.

8000:	learn: 180.2964850	test: 2373.9365505	best: 2373.9193209 (7996)	total: 1m 39s	remaining: 2m 28s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 2372.697617
bestIteration = 8606

Shrink model to first 8607 iterations.


,FA단순가중,FA시간가중,FA경기가중,FA시간경기가중,NONFA단순가중,NONFA시간가중,NONFA경기가중,NONFA시간경기가중
0,18832.2,18407.9,18842.6,18073.3,8482.9,8002.0,8386.9,8247.7
1,16296.6,16263.9,15960.8,16211.1,6446.2,6659.3,6564.8,6678.5


#### (2) 투수

In [10]:
pitcher_zscore = pd.DataFrame()

for pitcher_data, colname in zip(pitcher_datas, colnames):
    
    X, y = Xy_split(pitcher_data)
    
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X)
    
    tmp = pd.DataFrame([lgbm_model(X, y), cb_model(X, y)], columns=[colname])
    pitcher_zscore = pd.concat([pitcher_zscore, tmp], axis=1)
    
pitcher_zscore



 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[190]	valid_0's rmse: 11327.1


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[301]	valid_0's rmse: 16790.4


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[239]	valid_0's rmse: 18835.8


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[680]	valid_0's rmse: 13374.7


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[70]	valid_0's rmse: 12038.5
Learning rate set to 0.006285
0:	learn: 47195.99552

7000:	learn: 388.8711694	test: 14741.4056449	best: 14741.4056449 (7000)	total: 1m 19s	remaining: 2m 28s
8000:	learn: 259.9046009	test: 14730.6155051	best: 14730.5797495 (7981)	total: 1m 30s	remaining: 2m 16s
9000:	learn: 176.2990959	test: 14722.6485896	best: 14722.6485896 (9000)	total: 1m 41s	remaining: 2m 4s
10000:	learn: 117.5249219	test: 14716.8658124	best: 14716.8388628 (9996)	total: 1m 53s	remaining: 1m 53s
11000:	learn: 78.3503938	test: 14713.4934599	best: 14713.4934599 (11000)	total: 2m 5s	remaining: 1m 42s
12000:	learn: 53.0807769	test: 14711.6536243	best: 14711.6269769 (11976)	total: 2m 16s	remaining: 1m 31s
13000:	learn: 35.7835955	test: 14710.4471294	best: 14710.4390691 (12998)	total: 2m 27s	remaining: 1m 19s
14000:	learn: 24.4105678	test: 14709.7853475	best: 14709.7838091 (13948)	total: 2m 38s	remaining: 1m 7s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 14709.74586
bestIteration = 14079

Shrink model to first 14080 iterations.
Learning rate set to 0.0

1000:	learn: 7237.7431391	test: 15527.7353511	best: 15527.7353511 (1000)	total: 11.8s	remaining: 3m 44s
2000:	learn: 3648.6770178	test: 14361.2039021	best: 14360.4094730 (1997)	total: 23.9s	remaining: 3m 34s
3000:	learn: 2123.8531534	test: 14115.3499161	best: 14115.2923428 (2998)	total: 36s	remaining: 3m 23s
4000:	learn: 1345.3199776	test: 14033.2209206	best: 14033.2209206 (4000)	total: 47.6s	remaining: 3m 10s
5000:	learn: 892.8054278	test: 13994.3353335	best: 13994.3307004 (4999)	total: 58.9s	remaining: 2m 56s
6000:	learn: 580.8646665	test: 13971.6197790	best: 13971.5690833 (5995)	total: 1m 9s	remaining: 2m 43s
7000:	learn: 379.0971360	test: 13959.1827509	best: 13959.1827509 (7000)	total: 1m 21s	remaining: 2m 30s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 13953.16966
bestIteration = 7573

Shrink model to first 7574 iterations.
Learning rate set to 0.006287
0:	learn: 45907.7044643	test: 47753.8582329	best: 47753.8582329 (0)	total: 11.9ms	remaining: 3m 58s
1000:	



 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[176]	valid_0's rmse: 4665.8


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[220]	valid_0's rmse: 12044


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[43]	valid_0's rmse: 7314.54


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[22]	valid_0's rmse: 3799.42


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[84]	valid_0's rmse: 4490.26
Learning rate set to 0.007555
0:	learn: 11785.4887314	te

0:	learn: 9559.9630037	test: 16957.2948128	best: 16957.2948128 (0)	total: 16.6ms	remaining: 5m 32s
1000:	learn: 2625.3271503	test: 12972.7161430	best: 12972.7161430 (1000)	total: 12.2s	remaining: 3m 51s
2000:	learn: 1687.3451004	test: 12733.7473578	best: 12733.7428925 (1999)	total: 24s	remaining: 3m 35s
3000:	learn: 1203.4955946	test: 12676.5091637	best: 12676.2318541 (2980)	total: 36.2s	remaining: 3m 24s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 12673.00693
bestIteration = 3189

Shrink model to first 3190 iterations.
Learning rate set to 0.007556
0:	learn: 11688.3325217	test: 10304.1404263	best: 10304.1404263 (0)	total: 12.8ms	remaining: 4m 16s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 8110.923386
bestIteration = 287

Shrink model to first 288 iterations.
Learning rate set to 0.007556
0:	learn: 12132.4923581	test: 7988.1675493	best: 7988.1675493 (0)	total: 13.1ms	remaining: 4m 22s
Stopped by overfitting detector  (100 iterations wait)


,FA단순가중,FA시간가중,FA경기가중,FA시간경기가중,NONFA단순가중,NONFA시간가중,NONFA경기가중,NONFA시간경기가중
0,14473.3,14610.0,14472.2,15031.7,6462.8,6379.9,6424.1,6584.3
1,14219.2,13807.8,13901.2,13704.1,6780.2,6895.3,6775.6,6810.1


# 3. 표준화 (Min-Max-scaling)

In [11]:
from sklearn.preprocessing import MinMaxScaler

#### (1) 타자

In [12]:
hitter_minmaxscaling = pd.DataFrame()

for hitter_data, colname in zip(hitter_datas, colnames):
    
    X, y = Xy_split(hitter_data)
    
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X)
    
    tmp = pd.DataFrame([lgbm_model(X, y), cb_model(X, y)], columns=[colname])
    hitter_minmaxscaling = pd.concat([hitter_minmaxscaling, tmp], axis=1)
    
hitter_minmaxscaling



 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[44]	valid_0's rmse: 18915.8


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[98]	valid_0's rmse: 22838.6


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[52]	valid_0's rmse: 14754


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[323]	valid_0's rmse: 17225.7


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[62]	valid_0's rmse: 21617.8
Learning rate set to 0.007132
0:	learn: 50873.9113456	te

1000:	learn: 10027.4030776	test: 21920.7125245	best: 21920.7125245 (1000)	total: 12.2s	remaining: 3m 51s
2000:	learn: 6474.7266390	test: 20766.1822162	best: 20766.1822162 (2000)	total: 24.6s	remaining: 3m 41s
3000:	learn: 4354.3894734	test: 20421.3818751	best: 20421.3818751 (3000)	total: 37.3s	remaining: 3m 31s
4000:	learn: 3017.9000046	test: 20240.4062889	best: 20240.4062889 (4000)	total: 49.8s	remaining: 3m 19s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 20163.88274
bestIteration = 4720

Shrink model to first 4721 iterations.
Learning rate set to 0.007132
0:	learn: 51098.9979642	test: 48307.3680418	best: 48307.3680418 (0)	total: 14.3ms	remaining: 4m 45s
1000:	learn: 9912.1552398	test: 15193.4921446	best: 15193.3469123 (999)	total: 12.1s	remaining: 3m 49s
2000:	learn: 6343.6593171	test: 14402.8422764	best: 14402.0695222 (1995)	total: 24.1s	remaining: 3m 36s
3000:	learn: 4236.8271563	test: 14092.2337601	best: 14091.7971266 (2999)	total: 36.6s	remaining: 3m 27s
40

6000:	learn: 1331.7392023	test: 14695.7256061	best: 14695.3051642 (5990)	total: 1m 12s	remaining: 2m 49s
7000:	learn: 950.1895326	test: 14676.9311858	best: 14676.9311858 (7000)	total: 1m 25s	remaining: 2m 38s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 14663.70126
bestIteration = 7784

Shrink model to first 7785 iterations.
Learning rate set to 0.007133
0:	learn: 50790.4939739	test: 49688.1009491	best: 49688.1009491 (0)	total: 11.7ms	remaining: 3m 53s
1000:	learn: 10117.9677236	test: 18892.4207895	best: 18891.9564149 (999)	total: 12.1s	remaining: 3m 49s
2000:	learn: 6421.1538049	test: 17472.5797743	best: 17472.4920692 (1999)	total: 23.7s	remaining: 3m 33s
3000:	learn: 3944.9373301	test: 16998.8896863	best: 16998.3482256 (2999)	total: 35.8s	remaining: 3m 23s
4000:	learn: 2645.2184865	test: 16835.3448296	best: 16834.7928929 (3998)	total: 48.4s	remaining: 3m 13s
5000:	learn: 1802.7158273	test: 16779.9602185	best: 16779.6862813 (4993)	total: 1m	remaining: 3m 2s
6000:

1000:	learn: 2450.6403427	test: 7380.4309178	best: 7380.3978457 (999)	total: 11.7s	remaining: 3m 41s
2000:	learn: 1444.0767522	test: 6385.8132195	best: 6385.8132195 (2000)	total: 23.2s	remaining: 3m 28s
3000:	learn: 957.9088695	test: 6304.9964999	best: 6304.9964999 (3000)	total: 34.4s	remaining: 3m 14s
4000:	learn: 692.4308945	test: 6283.6810814	best: 6283.6810814 (4000)	total: 45.3s	remaining: 3m 1s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 6275.373627
bestIteration = 4688

Shrink model to first 4689 iterations.
Learning rate set to 0.007619
0:	learn: 14943.3670156	test: 13068.3398528	best: 13068.3398528 (0)	total: 10.4ms	remaining: 3m 27s
1000:	learn: 2006.5104749	test: 6830.0229069	best: 6830.0229069 (1000)	total: 10.7s	remaining: 3m 22s
2000:	learn: 1037.0023299	test: 6540.5332674	best: 6540.4433249 (1998)	total: 21.4s	remaining: 3m 12s
3000:	learn: 723.9292708	test: 6490.8521890	best: 6490.8521890 (3000)	total: 32.4s	remaining: 3m 3s
4000:	learn: 521.76598

Early stopping, best iteration is:
[11]	valid_0's rmse: 3198.32


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's rmse: 16398.9
[2000]	valid_0's rmse: 16280.4
[3000]	valid_0's rmse: 16276
[4000]	valid_0's rmse: 16275.9
Early stopping, best iteration is:
[4741]	valid_0's rmse: 16275.9


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[8]	valid_0's rmse: 3847.47
Learning rate set to 0.007619
0:	learn: 14139.0759123	test: 16294.6702244	best: 16294.6702244 (0)	total: 13.6ms	remaining: 4m 32s
1000:	learn: 2427.2539422	test: 8133.0519996	best: 8133.0519996 (1000)	total: 12.6s	remaining: 3m 58s
2000:	learn: 1453.5843656	test: 7086.4134970	best: 7086.2932481 (1998)	total: 25.3s	remaining: 3m 47s
3000:	learn: 928.2918191	test: 6975.5595404	best: 6975.5595404 (3000)	total: 37.7s	remaining: 3m 33s


Stopped by overfitting detector  (100 iterations wait)

bestTest = 2406.549373
bestIteration = 5098

Shrink model to first 5099 iterations.


,FA단순가중,FA시간가중,FA경기가중,FA시간경기가중,NONFA단순가중,NONFA시간가중,NONFA경기가중,NONFA시간경기가중
0,19070.4,18545.9,18827.0,18221.1,8003.2,8350.5,8341.1,8362.4
1,16315.1,16243.7,16030.4,16204.6,6446.1,6659.7,6564.7,6684.3


#### (2) 투수

In [13]:
pitcher_minmaxscaling = pd.DataFrame()

for pitcher_data, colname in zip(pitcher_datas, colnames):
    
    X, y = Xy_split(pitcher_data)
    
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X)
    
    tmp = pd.DataFrame([lgbm_model(X, y), cb_model(X, y)], columns=[colname])
    pitcher_minmaxscaling = pd.concat([pitcher_minmaxscaling, tmp], axis=1)
    
pitcher_minmaxscaling



 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[204]	valid_0's rmse: 11265.6


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[337]	valid_0's rmse: 17129.8


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[214]	valid_0's rmse: 18350.9


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[726]	valid_0's rmse: 13055.6


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[87]	valid_0's rmse: 11825.5
Learning rate set to 0.006285
0:	learn: 47195.99552

7000:	learn: 388.8711694	test: 14741.5644025	best: 14741.5644025 (7000)	total: 1m 19s	remaining: 2m 28s
8000:	learn: 259.9046009	test: 14730.7507262	best: 14730.7151141 (7981)	total: 1m 30s	remaining: 2m 15s
9000:	learn: 176.2990959	test: 14722.7749412	best: 14722.7749412 (9000)	total: 1m 41s	remaining: 2m 4s
10000:	learn: 117.5249219	test: 14716.9899828	best: 14716.9630432 (9996)	total: 1m 53s	remaining: 1m 53s
11000:	learn: 78.3503938	test: 14713.6155179	best: 14713.6155179 (11000)	total: 2m 5s	remaining: 1m 42s
12000:	learn: 53.0807769	test: 14711.7751311	best: 14711.7484924 (11976)	total: 2m 16s	remaining: 1m 31s
13000:	learn: 35.7835955	test: 14710.5679517	best: 14710.5598897 (12998)	total: 2m 27s	remaining: 1m 19s
14000:	learn: 24.4105678	test: 14709.9073397	best: 14709.9058168 (13948)	total: 2m 38s	remaining: 1m 7s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 14709.86785
bestIteration = 14079

Shrink model to first 14080 iterations.
Learning rate set to 0.0

1000:	learn: 7237.7431391	test: 15497.8664711	best: 15497.8664711 (1000)	total: 11.5s	remaining: 3m 38s
2000:	learn: 3648.6770178	test: 14273.4898626	best: 14272.6900268 (1997)	total: 23.3s	remaining: 3m 29s
3000:	learn: 2123.8531534	test: 14022.5257483	best: 14022.4636495 (2998)	total: 35.2s	remaining: 3m 19s
4000:	learn: 1345.3199776	test: 13938.7927248	best: 13938.7927248 (4000)	total: 46.7s	remaining: 3m 6s
5000:	learn: 892.8054278	test: 13899.4958884	best: 13899.4912434 (4999)	total: 57.7s	remaining: 2m 52s
6000:	learn: 580.8646665	test: 13876.4273031	best: 13876.3750548 (5995)	total: 1m 8s	remaining: 2m 40s
7000:	learn: 379.0971360	test: 13863.9196637	best: 13863.9196637 (7000)	total: 1m 20s	remaining: 2m 29s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 13857.96538
bestIteration = 7573

Shrink model to first 7574 iterations.
Learning rate set to 0.006287
0:	learn: 45907.7044643	test: 47753.8582329	best: 47753.8582329 (0)	total: 16.3ms	remaining: 5m 26s
1000:

Early stopping, best iteration is:
[19]	valid_0's rmse: 3881.54


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[82]	valid_0's rmse: 4309.53
Learning rate set to 0.007555
0:	learn: 11785.4887314	test: 9854.8231233	best: 9854.8231233 (0)	total: 11.2ms	remaining: 3m 44s
1000:	learn: 2775.2887444	test: 5356.9920368	best: 5356.5621322 (994)	total: 10.1s	remaining: 3m 11s
2000:	learn: 1694.9985691	test: 5144.1171703	best: 5143.6436869 (1987)	total: 20s	remaining: 2m 59s
3000:	learn: 1160.9892215	test: 5090.6185383	best: 5090.5409149 (2999)	total: 30.2s	remaining: 2m 50s
4000:	learn: 823.2108232	test: 5070.9228953	best: 5070.8514839 (3999)	total: 40.6s	remaining: 2m 42s
5000:	learn: 604.6714439	test: 5049.8800910	best: 5049.8800910 (5000)	total: 51.2s	remaining: 2m 33s
6000:	learn: 450.2445010	test: 5040.5791914	best: 5040.3541007 (5984)	total: 1m 1s	remaining: 2m 23s
Stopped by ov

3000:	learn: 1203.4955946	test: 12676.4955283	best: 12676.2181888 (2980)	total: 36.8s	remaining: 3m 28s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 12672.99291
bestIteration = 3189

Shrink model to first 3190 iterations.
Learning rate set to 0.007556
0:	learn: 11688.3325217	test: 10304.1404263	best: 10304.1404263 (0)	total: 19.9ms	remaining: 6m 37s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 8110.922374
bestIteration = 287

Shrink model to first 288 iterations.
Learning rate set to 0.007556
0:	learn: 12132.4923581	test: 7988.1675493	best: 7988.1675493 (0)	total: 13.2ms	remaining: 4m 23s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 3320.862733
bestIteration = 721

Shrink model to first 722 iterations.
Learning rate set to 0.007556
0:	learn: 11761.6473804	test: 9966.4785534	best: 9966.4785534 (0)	total: 14ms	remaining: 4m 40s
1000:	learn: 2791.8571466	test: 4792.5380575	best: 4792.5380575 (1000)	total: 12.5s	remaining: 3

,FA단순가중,FA시간가중,FA경기가중,FA시간경기가중,NONFA단순가중,NONFA시간가중,NONFA경기가중,NONFA시간경기가중
0,14325.5,14082.9,14494.4,14740.3,6438.7,6334.6,6516.6,6500.4
1,14187.5,13801.6,13882.8,13698.7,6710.5,6894.9,6775.5,6810.2


# 4. log변환

In [14]:
def lgbm_log_model(data_x, data_y):

    # lgbm 모델링
    lgb_params = {"objective" : "rmse",
                 "verbosity" : -1}

    performance = []

    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        X_train = data_x.iloc[tr_idx, :].values
        y_train = data_y[tr_idx].values

        X_valid = data_x.iloc[val_idx, :].values
        y_valid = data_y[val_idx].values

        lgb_dtrain = lgb.Dataset(data = X_train, label = y_train) 
        lgb_dvalid = lgb.Dataset(data = X_valid, label = y_valid) 

        lgb_model = lgb.train(lgb_params, lgb_dtrain, 20000, valid_sets=[lgb_dvalid], early_stopping_rounds=100, verbose_eval=1000)
        pred_valid = list(map(lambda x: 10 ** x, lgb_model.predict(X_valid)))
        rmse = mean_squared_error(list(map(lambda x: 10 ** x, y_valid)), pred_valid, squared=False)

        performance.append(rmse)

    # lgbm 성능종합
    cv_performance = round(np.mean(performance),1)

    return cv_performance

In [15]:
def cb_log_model(data_x, data_y):

    # cb 모델링
    cat_cols = []
    performance = []

    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        X_train = data_x.iloc[tr_idx, :]
        y_train = data_y[tr_idx]

        X_valid = data_x.iloc[val_idx, :]
        y_valid = data_y[val_idx]

        cb_dtrain = Pool(data=X_train, label=y_train, cat_features=cat_cols)
        cb_dvalid = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

        cb_model = CatBoostRegressor(iterations=20000, eval_metric='RMSE', loss_function='RMSE', verbose = 0)

        cb_model.fit(cb_dtrain, eval_set=cb_dvalid, early_stopping_rounds=100, verbose_eval=1000, use_best_model=True)
        pred_valid = list(map(lambda x: 10 ** x, cb_model.predict(X_valid)))
        rmse = mean_squared_error(list(map(lambda x: 10 ** x, y_valid)), pred_valid, squared=False)

        performance.append(rmse)
        
    # cb 성능종합
    cv_performance = round(np.mean(performance),1)

    return cv_performance

In [19]:
import math

#### (1) 타자

In [20]:
hitter_log = pd.DataFrame()

for hitter_data, colname in zip(hitter_datas, colnames):
    
    X, y = Xy_split(hitter_data)
    y = pd.Series(list(map(lambda x: math.log10(x), y)))
    
    tmp = pd.DataFrame([lgbm_log_model(X, y), cb_log_model(X, y)], columns=[colname])
    hitter_log = pd.concat([hitter_log, tmp], axis=1)
    
hitter_log

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[73]	valid_0's rmse: 0.123933
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[119]	valid_0's rmse: 0.156472
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[55]	valid_0's rmse: 0.139165
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[72]	valid_0's rmse: 0.135681
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[65]	valid_0's rmse: 0.138213
Learning rate set to 0.007132
0:	learn: 0.4756877	test: 0.4790060	best: 0.4790060 (0)	total: 8.31ms	remaining: 2m 46s
1000:	learn: 0.1144375	test: 0.1347612	best: 0.1347612 (1000)	total: 8.31s	remaining: 2m 37s
2000:	learn: 0.0814294	test: 0.1275806	best: 0.1275806 (2000)	total: 16.7s	remaining: 2m 30s
3000:	learn: 0.0582078	test: 0.1245512	best: 0.1245512 (3000

Early stopping, best iteration is:
[83]	valid_0's rmse: 0.137359
Learning rate set to 0.007132
0:	learn: 0.4756541	test: 0.4789536	best: 0.4789536 (0)	total: 22.5ms	remaining: 7m 29s
1000:	learn: 0.1124543	test: 0.1350316	best: 0.1350316 (1000)	total: 12.2s	remaining: 3m 52s
2000:	learn: 0.0793291	test: 0.1278704	best: 0.1278520 (1995)	total: 24.8s	remaining: 3m 43s
3000:	learn: 0.0573933	test: 0.1247623	best: 0.1247606 (2999)	total: 38.4s	remaining: 3m 37s
4000:	learn: 0.0403633	test: 0.1230725	best: 0.1230725 (4000)	total: 50.9s	remaining: 3m 23s
5000:	learn: 0.0292673	test: 0.1222335	best: 0.1222335 (5000)	total: 1m 3s	remaining: 3m 11s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.1219724027
bestIteration = 5429

Shrink model to first 5430 iterations.
Learning rate set to 0.007132
0:	learn: 0.4678691	test: 0.5087107	best: 0.5087107 (0)	total: 14ms	remaining: 4m 40s
1000:	learn: 0.1046901	test: 0.1612533	best: 0.1612533 (1000)	total: 12.3s	remaining: 3m 52s
20

1000:	learn: 0.0802357	test: 0.0927378	best: 0.0927378 (1000)	total: 11.7s	remaining: 3m 42s
2000:	learn: 0.0577334	test: 0.0878902	best: 0.0878825 (1998)	total: 23.2s	remaining: 3m 28s
3000:	learn: 0.0426601	test: 0.0860729	best: 0.0860654 (2997)	total: 34.4s	remaining: 3m 15s
4000:	learn: 0.0319335	test: 0.0853402	best: 0.0853241 (3990)	total: 45.4s	remaining: 3m 1s
5000:	learn: 0.0244635	test: 0.0848212	best: 0.0848210 (4996)	total: 56.6s	remaining: 2m 49s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.08473662941
bestIteration = 5237

Shrink model to first 5238 iterations.
Learning rate set to 0.007619
0:	learn: 0.2858952	test: 0.2995039	best: 0.2995039 (0)	total: 12.7ms	remaining: 4m 14s
1000:	learn: 0.0758121	test: 0.1148090	best: 0.1148086 (999)	total: 12s	remaining: 3m 47s
2000:	learn: 0.0542529	test: 0.1109328	best: 0.1109326 (1999)	total: 23.3s	remaining: 3m 29s
3000:	learn: 0.0400679	test: 0.1097737	best: 0.1097737 (3000)	total: 34.4s	remaining: 3m 14s


Early stopping, best iteration is:
[53]	valid_0's rmse: 0.10566
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's rmse: 0.112024
[2000]	valid_0's rmse: 0.111914
[3000]	valid_0's rmse: 0.111909
[4000]	valid_0's rmse: 0.111909
Early stopping, best iteration is:
[4225]	valid_0's rmse: 0.111909
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[152]	valid_0's rmse: 0.100973
Learning rate set to 0.007619
0:	learn: 0.2912063	test: 0.2787521	best: 0.2787521 (0)	total: 14ms	remaining: 4m 40s
1000:	learn: 0.0800050	test: 0.0926359	best: 0.0926314 (999)	total: 13.3s	remaining: 4m 12s
2000:	learn: 0.0573986	test: 0.0869009	best: 0.0869009 (2000)	total: 26.4s	remaining: 3m 57s
3000:	learn: 0.0419392	test: 0.0845511	best: 0.0845511 (3000)	total: 38.9s	remaining: 3m 40s
4000:	learn: 0.0312866	test: 0.0836469	best: 0.0836469 (4000)	total: 51s	remaining: 3m 23s
5000:	learn: 0.0238708	test: 0.0831212	best: 0.0831211 (4999)	tota

,FA단순가중,FA시간가중,FA경기가중,FA시간경기가중,NONFA단순가중,NONFA시간가중,NONFA경기가중,NONFA시간경기가중
0,17946.5,17834.7,17520.0,17957.2,8452.4,8717.6,7978.3,7963.4
1,18035.8,18034.6,17452.0,17865.0,7040.2,7053.1,6877.5,6760.7


#### (2) 투수

In [21]:
pitcher_log = pd.DataFrame()

for pitcher_data, colname in zip(pitcher_datas, colnames):
    
    X, y = Xy_split(pitcher_data)
    y = pd.Series(list(map(lambda x: math.log10(x), y)))
    
    tmp = pd.DataFrame([lgbm_log_model(X, y), cb_log_model(X, y)], columns=[colname])
    pitcher_log = pd.concat([pitcher_log, tmp], axis=1)
    
pitcher_log

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[702]	valid_0's rmse: 0.140025
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's rmse: 0.167374
Early stopping, best iteration is:
[925]	valid_0's rmse: 0.167364
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[432]	valid_0's rmse: 0.147693
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[790]	valid_0's rmse: 0.147322
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[56]	valid_0's rmse: 0.141469
Learning rate set to 0.006285
0:	learn: 0.4582012	test: 0.4443191	best: 0.4443191 (0)	total: 12.8ms	remaining: 4m 16s
1000:	learn: 0.1017026	test: 0.1371095	best: 0.1371095 (1000)	total: 9.35s	remaining: 2m 57s
2000:	learn: 0.0587693	test: 0.1274751	best: 0.1274682 (1997)	total: 18.9s	remaining: 2m 49s
Stopped by overfitting de

Early stopping, best iteration is:
[1531]	valid_0's rmse: 0.156533
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[306]	valid_0's rmse: 0.140597
Learning rate set to 0.006285
0:	learn: 0.4578847	test: 0.4439729	best: 0.4439729 (0)	total: 13.5ms	remaining: 4m 30s
1000:	learn: 0.1014682	test: 0.1393127	best: 0.1393127 (1000)	total: 12.4s	remaining: 3m 55s
2000:	learn: 0.0591331	test: 0.1276784	best: 0.1276784 (2000)	total: 24.4s	remaining: 3m 39s
3000:	learn: 0.0359716	test: 0.1257718	best: 0.1257548 (2985)	total: 35.9s	remaining: 3m 23s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.1253210974
bestIteration = 3491

Shrink model to first 3492 iterations.
Learning rate set to 0.006285
0:	learn: 0.4566746	test: 0.4485851	best: 0.4485851 (0)	total: 11.4ms	remaining: 3m 47s
1000:	learn: 0.0956018	test: 0.1669929	best: 0.1669929 (1000)	total: 11.6s	remaining: 3m 39s
2000:	learn: 0.0554863	test: 0.1569807	best: 0.1569807 (

Early stopping, best iteration is:
[158]	valid_0's rmse: 0.124353
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[52]	valid_0's rmse: 0.144887
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[77]	valid_0's rmse: 0.116183
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[124]	valid_0's rmse: 0.116432
Learning rate set to 0.007555
0:	learn: 0.3121946	test: 0.3244178	best: 0.3244178 (0)	total: 23.8ms	remaining: 7m 55s
1000:	learn: 0.0967784	test: 0.1266414	best: 0.1266414 (1000)	total: 10.6s	remaining: 3m 21s
2000:	learn: 0.0683613	test: 0.1199822	best: 0.1199629 (1989)	total: 21.4s	remaining: 3m 12s
3000:	learn: 0.0499916	test: 0.1184474	best: 0.1184474 (3000)	total: 32.6s	remaining: 3m 4s
4000:	learn: 0.0367791	test: 0.1179046	best: 0.1178996 (3975)	total: 43.6s	remaining: 2m 54s
Stopped by overfitting detector  (100 iterations wait)

bestTes

3000:	learn: 0.0494551	test: 0.1199176	best: 0.1199081 (2964)	total: 38.8s	remaining: 3m 39s
4000:	learn: 0.0361807	test: 0.1192056	best: 0.1192032 (3999)	total: 51.2s	remaining: 3m 24s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.1188209582
bestIteration = 4803

Shrink model to first 4804 iterations.
Learning rate set to 0.007555
0:	learn: 0.3141072	test: 0.3170325	best: 0.3170325 (0)	total: 13.5ms	remaining: 4m 30s
1000:	learn: 0.0968857	test: 0.1336323	best: 0.1336317 (998)	total: 13.2s	remaining: 4m 10s
2000:	learn: 0.0699022	test: 0.1260244	best: 0.1260174 (1999)	total: 25.7s	remaining: 3m 51s
3000:	learn: 0.0502088	test: 0.1230084	best: 0.1229952 (2989)	total: 38.9s	remaining: 3m 40s
4000:	learn: 0.0372876	test: 0.1218613	best: 0.1218602 (3999)	total: 51.7s	remaining: 3m 26s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.1217213015
bestIteration = 4210

Shrink model to first 4211 iterations.
Learning rate set to 0.007556
0:	learn: 0.3

,FA단순가중,FA시간가중,FA경기가중,FA시간경기가중,NONFA단순가중,NONFA시간가중,NONFA경기가중,NONFA시간경기가중
0,15979.6,15695.2,15521.3,15800.2,6532.6,6496.9,6467.5,6372.3
1,16284.7,16664.2,16572.0,16484.0,6356.2,6346.8,6341.9,6360.2


# 5. Feature Selection (RFE)

#### (1) 순위도출

In [28]:
def get_lgbm_RFE_rank(dataset):

    # train, test 분할
    X, y = Xy_split(dataset)

    X, X_test, y, y_test = train_test_split(X, y, test_size = 0.25, shuffle = True, random_state = 42)

    X = X.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y = y.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)


    # RFE 적용
    lgb_params = {"objective" : "rmse",
                 "verbosity" : -1}

    remove_cols = []

    while True:

        # 변수명 정의
        column_names = list(X.columns)
        print(f'남은변수개수: {len(column_names)}')

        # lgbm 모델링
        lgb_dtrain = lgb.Dataset(data = X, label = y) 
        lgb_dvalid = lgb.Dataset(data = X_test, label = y_test) 

        lgb_model = lgb.train(lgb_params, lgb_dtrain, 20000, valid_sets=[lgb_dvalid], early_stopping_rounds=100, verbose_eval=1000)

        # 가장 낮은 변수들 중 1개 삭제
        remove_col = [col for col, importance in zip(column_names, lgb_model.feature_importance()) if importance == min(lgb_model.feature_importance())][-1]
        remove_cols.append(remove_col)
        X.drop(remove_col, axis=1, inplace=True)
        X_test.drop(remove_col, axis=1, inplace=True)

        # while문 종료조건
        if X.shape[1] == 1:
            remove_cols.append(X.columns[0])
            break

    # 데이터프레임으로 생성
    result = pd.DataFrame({'변수명':remove_cols,
                           'lgbm순위':[i for i in range(dataset.shape[1]-1, 0, -1)]})

    return result

In [29]:
def get_cb_RFE_rank(dataset):

    # train, test 분할
    X, y = Xy_split(dataset)

    X, X_test, y, y_test = train_test_split(X, y, test_size = 0.25, shuffle = True, random_state = 42)

    X = X.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y = y.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    # RFE 적용
    remove_cols = []

    while True:

        # 변수명 정의
        column_names = list(X.columns)
        print(f'남은변수개수: {len(column_names)}')

        # cb 모델링
        cat_cols = []
        cb_dtrain = Pool(data=X, label=y, cat_features=cat_cols)
        cb_dvalid = Pool(data=X_test, label=y_test, cat_features=cat_cols)

        cb_model = CatBoostRegressor(iterations=20000, eval_metric='RMSE', loss_function='RMSE', verbose = 0)
        cb_model.fit(cb_dtrain, eval_set=cb_dvalid, early_stopping_rounds=100, verbose_eval=1000, use_best_model=True)

        # 가장 낮은 변수들 중 1개 삭제
        remove_col = [col for col, importance in zip(column_names, cb_model.feature_importances_) if importance == min(cb_model.feature_importances_)][-1]
        remove_cols.append(remove_col)
        X.drop(remove_col, axis=1, inplace=True)
        X_test.drop(remove_col, axis=1, inplace=True)

        # while문 종료조건
        if X.shape[1] == 1:
            remove_cols.append(X.columns[0])
            break

    # 데이터프레임으로 생성
    result = pd.DataFrame({'변수명':remove_cols,
                           'cb순위':[i for i in range(dataset.shape[1]-1, 0, -1)]})
    
    return result

In [45]:
hitter_lgbm_rfe_score[hitter_lgbm_rfe_score['변수명'] == 'FA여부']

,변수명,FA단순가중,FA시간가중,FA경기가중,FA시간경기가중,NONFA단순가중,NONFA시간가중,NONFA경기가중,NONFA시간경기가중
106,FA여부,39,48,40,60,134,131,128,130


In [32]:
cnt = 1

for hitter_data, colname in zip(hitter_datas, colnames):
    
    tmp = get_lgbm_RFE_rank(hitter_data)
    tmp.columns = ['변수명', colname]
    
    if cnt == 1:
        hitter_lgbm_rfe_score = tmp.copy()
    else:
        hitter_lgbm_rfe_score = pd.merge(hitter_lgbm_rfe_score, tmp, how='left', on='변수명')
        
    cnt += 1
    
hitter_lgbm_rfe_score

남은변수개수: 145
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[82]	valid_0's rmse: 21951.1
남은변수개수: 144
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[82]	valid_0's rmse: 21951.1
남은변수개수: 143
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[82]	valid_0's rmse: 21951.1
남은변수개수: 142
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[82]	valid_0's rmse: 21951.1
남은변수개수: 141
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[82]	valid_0's rmse: 21951.1
남은변수개수: 140
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[82]	valid_0's rmse: 21951.1
남은변수개수: 139
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[82]	valid_0's rmse: 21951.1
남은변수개수: 138
Training until validat

Early stopping, best iteration is:
[63]	valid_0's rmse: 21650.6
남은변수개수: 85
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[77]	valid_0's rmse: 21617.8
남은변수개수: 84
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[61]	valid_0's rmse: 21789
남은변수개수: 83
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[69]	valid_0's rmse: 22061.6
남은변수개수: 82
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[69]	valid_0's rmse: 22116.8
남은변수개수: 81
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[72]	valid_0's rmse: 22004.1
남은변수개수: 80
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[154]	valid_0's rmse: 22202.3
남은변수개수: 79
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[80]	va

남은변수개수: 25
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[124]	valid_0's rmse: 22864.6
남은변수개수: 24
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[51]	valid_0's rmse: 23746.1
남은변수개수: 23
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[55]	valid_0's rmse: 23795.8
남은변수개수: 22
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[70]	valid_0's rmse: 22421.8
남은변수개수: 21
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[85]	valid_0's rmse: 22441.4
남은변수개수: 20
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[91]	valid_0's rmse: 22461.7
남은변수개수: 19
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[69]	valid_0's rmse: 22599.9
남은변수개수: 18
Training until validation sco

Early stopping, best iteration is:
[127]	valid_0's rmse: 19919.2
남은변수개수: 109
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[112]	valid_0's rmse: 19924
남은변수개수: 108
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[164]	valid_0's rmse: 19913.4
남은변수개수: 107
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[127]	valid_0's rmse: 19981.7
남은변수개수: 106
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[117]	valid_0's rmse: 19815.3
남은변수개수: 105
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[147]	valid_0's rmse: 20139.7
남은변수개수: 104
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[184]	valid_0's rmse: 20298.3
남은변수개수: 103
Training until validation scores don't improve for 100 rounds
Early stopping, best iteratio

Early stopping, best iteration is:
[503]	valid_0's rmse: 20801
남은변수개수: 49
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[835]	valid_0's rmse: 22130.8
남은변수개수: 48
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's rmse: 22850.4
[2000]	valid_0's rmse: 22844.5
Early stopping, best iteration is:
[2428]	valid_0's rmse: 22844.5
남은변수개수: 47
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[428]	valid_0's rmse: 26011.7
남은변수개수: 46
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[125]	valid_0's rmse: 26331.3
남은변수개수: 45
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[302]	valid_0's rmse: 26025.3
남은변수개수: 44
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[107]	valid_0's rmse: 25853
남은변수개수: 43
Training until validation scores don'

...

Early stopping, best iteration is:
[93]	valid_0's rmse: 9642.17
남은변수개수: 127
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[93]	valid_0's rmse: 9642.17
남은변수개수: 126
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[93]	valid_0's rmse: 9642.17
남은변수개수: 125
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[93]	valid_0's rmse: 9642.17
남은변수개수: 124
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[99]	valid_0's rmse: 9820.76
남은변수개수: 123
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[99]	valid_0's rmse: 9820.76
남은변수개수: 122
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[99]	valid_0's rmse: 9820.76
남은변수개수: 121
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:

Early stopping, best iteration is:
[82]	valid_0's rmse: 9789.79
남은변수개수: 67
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[78]	valid_0's rmse: 9852.31
남은변수개수: 66
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[75]	valid_0's rmse: 9852.32
남은변수개수: 65
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[100]	valid_0's rmse: 9809.37
남은변수개수: 64
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[80]	valid_0's rmse: 9870.05
남은변수개수: 63
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[86]	valid_0's rmse: 9931.56
남은변수개수: 62
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[73]	valid_0's rmse: 9831.98
남은변수개수: 61
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[81]	

Early stopping, best iteration is:
[82]	valid_0's rmse: 9860.33
남은변수개수: 92
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[103]	valid_0's rmse: 9832.01
남은변수개수: 91
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[110]	valid_0's rmse: 9930.07
남은변수개수: 90
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[85]	valid_0's rmse: 10007.2
남은변수개수: 89
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[818]	valid_0's rmse: 9311.41
남은변수개수: 88
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[128]	valid_0's rmse: 9855.59
남은변수개수: 87
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[112]	valid_0's rmse: 9738.18
남은변수개수: 86
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[

남은변수개수: 34
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[152]	valid_0's rmse: 9485.6
남은변수개수: 33
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[126]	valid_0's rmse: 9401.27
남은변수개수: 32
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's rmse: 8459.09
Early stopping, best iteration is:
[1767]	valid_0's rmse: 8419.07
남은변수개수: 31
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[161]	valid_0's rmse: 9321.53
남은변수개수: 30
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's rmse: 8358.24
Early stopping, best iteration is:
[1478]	valid_0's rmse: 8337.89
남은변수개수: 29
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[184]	valid_0's rmse: 9180.33
남은변수개수: 28
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[

,변수명,FA단순가중,FA시간가중,FA경기가중,FA시간경기가중,NONFA단순가중,NONFA시간가중,NONFA경기가중,NONFA시간경기가중
0,5년평균도루저지율(수비),145,145,117,117,131,121,102,96
1,5년평균경기(주루),144,144,145,145,145,145,144,145
2,3차FA여부,143,143,144,144,144,144,143,144
3,포지션(수비)_포수,142,142,143,143,141,141,140,141
4,포지션(수비)_중견수,141,127,142,142,132,140,139,140
...,...,...,...,...,...,...,...,...,...
140,뉴스개수(누적정규화),5,4,3,8,4,3,5,3
141,5년평균뜬공,4,14,2,22,34,26,73,52
142,누적타석,3,2,4,1,20,13,17,8
143,5년평균연봉,2,3,1,2,5,5,1,1


In [35]:
cnt = 1

for pitcher_data, colname in zip(pitcher_datas, colnames):
    
    tmp = get_lgbm_RFE_rank(pitcher_data)
    tmp.columns = ['변수명', colname]
    
    if cnt == 1:
        pitcher_lgbm_rfe_score = tmp.copy()
    else:
        pitcher_lgbm_rfe_score = pd.merge(pitcher_lgbm_rfe_score, tmp, how='left', on='변수명')
        
    cnt += 1
    
pitcher_lgbm_rfe_score

남은변수개수: 143
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[366]	valid_0's rmse: 11378.9
남은변수개수: 142
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[366]	valid_0's rmse: 11378.9
남은변수개수: 141
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[366]	valid_0's rmse: 11378.9
남은변수개수: 140
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[366]	valid_0's rmse: 11378.9
남은변수개수: 139
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[366]	valid_0's rmse: 11378.9
남은변수개수: 138
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[366]	valid_0's rmse: 11378.9
남은변수개수: 137
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[366]	valid_0's rmse: 11378.9
남은변수개수: 136
Training until 

Early stopping, best iteration is:
[430]	valid_0's rmse: 11081.3
남은변수개수: 83
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[316]	valid_0's rmse: 11429.2
남은변수개수: 82
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[335]	valid_0's rmse: 11353
남은변수개수: 81
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[360]	valid_0's rmse: 11251.2
남은변수개수: 80
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[310]	valid_0's rmse: 11277.3
남은변수개수: 79
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[291]	valid_0's rmse: 11227.1
남은변수개수: 78
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[311]	valid_0's rmse: 11213.4
남은변수개수: 77
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[

[30]	valid_0's rmse: 22085.7
남은변수개수: 22
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[39]	valid_0's rmse: 21982.9
남은변수개수: 21
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[33]	valid_0's rmse: 21454.5
남은변수개수: 20
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[35]	valid_0's rmse: 21230.3
남은변수개수: 19
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[36]	valid_0's rmse: 20923.8
남은변수개수: 18
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[70]	valid_0's rmse: 20272.5
남은변수개수: 17
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[74]	valid_0's rmse: 20452.5
남은변수개수: 16
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[40]	valid_0's rmse: 20958.9
남은변수개수: 15
T

Early stopping, best iteration is:
[302]	valid_0's rmse: 12077.6
남은변수개수: 104
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[292]	valid_0's rmse: 12072.3
남은변수개수: 103
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[277]	valid_0's rmse: 12184
남은변수개수: 102
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[279]	valid_0's rmse: 12059.4
남은변수개수: 101
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[291]	valid_0's rmse: 12037.2
남은변수개수: 100
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[245]	valid_0's rmse: 12302.6
남은변수개수: 99
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[251]	valid_0's rmse: 12437.8
남은변수개수: 98
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration 

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[130]	valid_0's rmse: 18257.2
남은변수개수: 43
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[64]	valid_0's rmse: 19349
남은변수개수: 42
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[94]	valid_0's rmse: 20265.9
남은변수개수: 41
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[57]	valid_0's rmse: 19839.7
남은변수개수: 40
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[77]	valid_0's rmse: 19501.3
남은변수개수: 39
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[102]	valid_0's rmse: 19867.6
남은변수개수: 38
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[47]	valid_0's rmse: 19558
남은변수개수: 37
Training until validation scores don't impr

...

Early stopping, best iteration is:
[140]	valid_0's rmse: 8704.69
남은변수개수: 94
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[114]	valid_0's rmse: 8756.78
남은변수개수: 93
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[133]	valid_0's rmse: 8566.06
남은변수개수: 92
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[156]	valid_0's rmse: 8546.89
남은변수개수: 91
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[134]	valid_0's rmse: 8644.71
남은변수개수: 90
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[167]	valid_0's rmse: 8571.02
남은변수개수: 89
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[143]	valid_0's rmse: 8555.65
남은변수개수: 88
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:

Early stopping, best iteration is:
[96]	valid_0's rmse: 8785.69
남은변수개수: 34
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[141]	valid_0's rmse: 8773.13
남은변수개수: 33
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[321]	valid_0's rmse: 8579.52
남은변수개수: 32
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[255]	valid_0's rmse: 8638.78
남은변수개수: 31
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[268]	valid_0's rmse: 8602.26
남은변수개수: 30
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[228]	valid_0's rmse: 8640.66
남은변수개수: 29
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[337]	valid_0's rmse: 8700.61
남은변수개수: 28
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:


Early stopping, best iteration is:
[91]	valid_0's rmse: 8758.81
남은변수개수: 116
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[188]	valid_0's rmse: 8662.41
남은변수개수: 115
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[116]	valid_0's rmse: 8696.36
남은변수개수: 114
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[163]	valid_0's rmse: 8641.27
남은변수개수: 113
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[95]	valid_0's rmse: 8640.06
남은변수개수: 112
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[95]	valid_0's rmse: 8640.06
남은변수개수: 111
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[104]	valid_0's rmse: 8592.78
남은변수개수: 110
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration

Early stopping, best iteration is:
[167]	valid_0's rmse: 8678.27
남은변수개수: 56
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[100]	valid_0's rmse: 8614
남은변수개수: 55
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[104]	valid_0's rmse: 8699.05
남은변수개수: 54
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[136]	valid_0's rmse: 8625.35
남은변수개수: 53
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[172]	valid_0's rmse: 8702.17
남은변수개수: 52
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[92]	valid_0's rmse: 8794.06
남은변수개수: 51
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[126]	valid_0's rmse: 8794.86
남은변수개수: 50
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[10

,변수명,FA단순가중,FA시간가중,FA경기가중,FA시간경기가중,NONFA단순가중,NONFA시간가중,NONFA경기가중,NONFA시간경기가중
0,5년평균포지션 엔트로피,143,143,143,143,143,143,143,143
1,2차FA여부,142,141,142,142,142,142,142,142
2,팀명_현대,141,140,141,141,130,140,140,140
3,팀명_삼성,140,137,139,139,110,109,106,100
4,팀명_SK,139,120,125,122,129,128,138,138
...,...,...,...,...,...,...,...,...,...
138,5년평균세이브기회,5,3,3,1,8,4,5,20
139,누적이닝,4,6,5,4,6,7,6,10
140,5년평균뉴스개수(연도별정규화),3,4,11,9,7,6,1,3
141,뉴스개수(누적정규화),2,2,1,2,13,9,12,17


In [36]:
cnt = 1

for hitter_data, colname in zip(hitter_datas, colnames):
    
    tmp = get_cb_RFE_rank(hitter_data)
    tmp.columns = ['변수명', colname]
    
    if cnt == 1:
        hitter_cb_rfe_score = tmp.copy()
    else:
        hitter_cb_rfe_score = pd.merge(hitter_cb_rfe_score, tmp, how='left', on='변수명')
        
    cnt += 1
    
hitter_cb_rfe_score

남은변수개수: 145
Learning rate set to 0.007045
0:	learn: 48734.2728413	test: 55650.5370306	best: 55650.5370306 (0)	total: 16.2ms	remaining: 5m 24s
1000:	learn: 10491.9467770	test: 21326.5970070	best: 21324.7745120 (996)	total: 11.3s	remaining: 3m 33s
2000:	learn: 6766.8105623	test: 20379.5935504	best: 20379.5935504 (2000)	total: 22.2s	remaining: 3m 19s
3000:	learn: 4369.6155667	test: 20048.8924650	best: 20048.8924650 (3000)	total: 33.2s	remaining: 3m 8s
4000:	learn: 2966.2730251	test: 19862.9797433	best: 19862.9405243 (3998)	total: 44.4s	remaining: 2m 57s
5000:	learn: 2047.5256468	test: 19758.5050746	best: 19758.4395314 (4999)	total: 55.8s	remaining: 2m 47s
6000:	learn: 1415.7068080	test: 19717.2861922	best: 19717.2861922 (6000)	total: 1m 7s	remaining: 2m 36s
7000:	learn: 1014.8275449	test: 19694.1005633	best: 19694.1005633 (7000)	total: 1m 17s	remaining: 2m 24s
8000:	learn: 732.4841255	test: 19675.6358640	best: 19675.5687807 (7997)	total: 1m 28s	remaining: 2m 13s
9000:	learn: 543.3288381	t

2000:	learn: 6794.8509104	test: 20015.8140552	best: 20015.8140552 (2000)	total: 23.3s	remaining: 3m 29s
3000:	learn: 4433.9919408	test: 19633.0214255	best: 19632.4196023 (2994)	total: 34.6s	remaining: 3m 16s
4000:	learn: 3064.1842658	test: 19460.7737523	best: 19460.4173448 (3998)	total: 45.6s	remaining: 3m 2s
5000:	learn: 2096.4994367	test: 19370.6677005	best: 19370.6677005 (5000)	total: 56.4s	remaining: 2m 49s
6000:	learn: 1433.6183284	test: 19326.9676481	best: 19326.8982825 (5999)	total: 1m 7s	remaining: 2m 36s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 19307.65171
bestIteration = 6565

Shrink model to first 6566 iterations.
남은변수개수: 137
Learning rate set to 0.007045
0:	learn: 48750.4185798	test: 55679.9476561	best: 55679.9476561 (0)	total: 19.8ms	remaining: 6m 36s
1000:	learn: 10450.8496879	test: 21376.6526067	best: 21376.6526067 (1000)	total: 11.5s	remaining: 3m 38s
2000:	learn: 6747.3680581	test: 20446.5136148	best: 20446.5136148 (2000)	total: 23.2s	remainin

4000:	learn: 3121.0046286	test: 19833.5253192	best: 19833.1279004 (3999)	total: 40.6s	remaining: 2m 42s
5000:	learn: 2137.1777012	test: 19739.1776289	best: 19738.3349974 (4996)	total: 50.6s	remaining: 2m 31s
6000:	learn: 1467.5081481	test: 19694.9938222	best: 19694.3985795 (5993)	total: 1m	remaining: 2m 21s
7000:	learn: 1060.2064267	test: 19668.9435888	best: 19668.9186592 (6986)	total: 1m 11s	remaining: 2m 12s
8000:	learn: 773.9818592	test: 19653.1145233	best: 19653.1145233 (8000)	total: 1m 21s	remaining: 2m 2s
9000:	learn: 578.4413286	test: 19643.9887750	best: 19643.8250192 (8968)	total: 1m 32s	remaining: 1m 52s
10000:	learn: 436.8417747	test: 19637.6182844	best: 19637.6127238 (9999)	total: 1m 42s	remaining: 1m 42s
11000:	learn: 329.5822726	test: 19633.7014815	best: 19633.5029941 (10931)	total: 1m 51s	remaining: 1m 31s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 19633.50299
bestIteration = 10931

Shrink model to first 10932 iterations.
남은변수개수: 129
Learning rate 

3000:	learn: 4503.5707664	test: 20088.5167958	best: 20088.5167958 (3000)	total: 29s	remaining: 2m 44s
4000:	learn: 3006.9191061	test: 19898.3639561	best: 19898.3639561 (4000)	total: 39.1s	remaining: 2m 36s
5000:	learn: 2041.0205014	test: 19797.1841217	best: 19797.1841217 (5000)	total: 49.2s	remaining: 2m 27s
6000:	learn: 1428.8187597	test: 19747.9119886	best: 19747.7306594 (5994)	total: 58.9s	remaining: 2m 17s
7000:	learn: 1002.7997422	test: 19717.4239264	best: 19717.4239264 (7000)	total: 1m 8s	remaining: 2m 7s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 19706.04238
bestIteration = 7416

Shrink model to first 7417 iterations.
남은변수개수: 121
Learning rate set to 0.007045
0:	learn: 48743.7977956	test: 55663.6212542	best: 55663.6212542 (0)	total: 8.97ms	remaining: 2m 59s
1000:	learn: 10442.7764516	test: 21120.3392659	best: 21120.3392659 (1000)	total: 9.74s	remaining: 3m 4s
2000:	learn: 6795.3011336	test: 20094.1569937	best: 20092.8303702 (1986)	total: 19.5s	remaining: 

Stopped by overfitting detector  (100 iterations wait)

bestTest = 19953.25329
bestIteration = 6630

Shrink model to first 6631 iterations.
남은변수개수: 113
Learning rate set to 0.007045
0:	learn: 48752.0781284	test: 55667.9663430	best: 55667.9663430 (0)	total: 18.7ms	remaining: 6m 13s
1000:	learn: 10442.2390416	test: 21179.9789779	best: 21179.9789779 (1000)	total: 9.51s	remaining: 3m
2000:	learn: 6865.9701693	test: 20282.3298266	best: 20282.3298266 (2000)	total: 18.7s	remaining: 2m 47s
3000:	learn: 4471.7801349	test: 19910.0477401	best: 19910.0477401 (3000)	total: 28.1s	remaining: 2m 38s
4000:	learn: 2956.4121927	test: 19758.2822404	best: 19758.2822404 (4000)	total: 37.8s	remaining: 2m 31s
5000:	learn: 1979.7529395	test: 19664.0040767	best: 19663.4366684 (4974)	total: 47.6s	remaining: 2m 22s
6000:	learn: 1373.2244634	test: 19629.4370109	best: 19629.3395424 (5986)	total: 57.4s	remaining: 2m 13s
7000:	learn: 982.9959292	test: 19608.7920833	best: 19608.7062175 (6980)	total: 1m 6s	remaining: 2

...

3000:	learn: 941.1522250	test: 5591.4385652	best: 5591.4385652 (3000)	total: 20.7s	remaining: 1m 57s
4000:	learn: 681.3923581	test: 5563.1661855	best: 5563.1661855 (4000)	total: 27.3s	remaining: 1m 49s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 5556.835238
bestIteration = 4751

Shrink model to first 4752 iterations.
남은변수개수: 52
Learning rate set to 0.007527
0:	learn: 14437.7104289	test: 15037.7746585	best: 15037.7746585 (0)	total: 6.4ms	remaining: 2m 7s
1000:	learn: 2417.6783815	test: 6529.8928467	best: 6529.8928467 (1000)	total: 6.76s	remaining: 2m 8s
2000:	learn: 1434.6229311	test: 5691.0468951	best: 5691.0468951 (2000)	total: 13.6s	remaining: 2m 2s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 5629.414441
bestIteration = 2472

Shrink model to first 2473 iterations.
남은변수개수: 51
Learning rate set to 0.007527
0:	learn: 14427.9469713	test: 15037.4988155	best: 15037.4988155 (0)	total: 7.14ms	remaining: 2m 22s
1000:	learn: 2410.1733446	test: 6615

Stopped by overfitting detector  (100 iterations wait)

bestTest = 5708.977215
bestIteration = 3260

Shrink model to first 3261 iterations.
남은변수개수: 38
Learning rate set to 0.007527
0:	learn: 14441.5894794	test: 15076.3534717	best: 15076.3534717 (0)	total: 13.1ms	remaining: 4m 21s
1000:	learn: 2418.6918739	test: 6560.5114979	best: 6560.5114861 (999)	total: 5.68s	remaining: 1m 47s
2000:	learn: 1489.2732051	test: 5754.3530445	best: 5754.3530445 (2000)	total: 11.1s	remaining: 1m 39s
3000:	learn: 1002.3993444	test: 5685.0455313	best: 5684.9220759 (2991)	total: 16.5s	remaining: 1m 33s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 5683.793704
bestIteration = 3076

Shrink model to first 3077 iterations.
남은변수개수: 37
Learning rate set to 0.007527
0:	learn: 14433.1004431	test: 15038.4459761	best: 15038.4459761 (0)	total: 5.51ms	remaining: 1m 50s
1000:	learn: 2427.7955457	test: 6627.5121921	best: 6627.5121921 (1000)	total: 5.26s	remaining: 1m 39s
2000:	learn: 1521.2141225	test:

Stopped by overfitting detector  (100 iterations wait)

bestTest = 7018.080046
bestIteration = 2100

Shrink model to first 2101 iterations.
남은변수개수: 21
Learning rate set to 0.007527
0:	learn: 14440.6613792	test: 15059.4217264	best: 15059.4217264 (0)	total: 4.2ms	remaining: 1m 23s
1000:	learn: 2358.7994791	test: 7463.3550755	best: 7463.3550755 (1000)	total: 3.3s	remaining: 1m 2s
2000:	learn: 1521.3758257	test: 7147.3311970	best: 7147.3311970 (2000)	total: 6.49s	remaining: 58.4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 7145.458891
bestIteration = 2031

Shrink model to first 2032 iterations.
남은변수개수: 20
Learning rate set to 0.007527
0:	learn: 14432.2030389	test: 15076.3228953	best: 15076.3228953 (0)	total: 4.05ms	remaining: 1m 20s
1000:	learn: 2360.1892126	test: 7321.8681728	best: 7321.8681728 (1000)	total: 3.05s	remaining: 58s
2000:	learn: 1520.0399011	test: 7018.6496052	best: 7018.6104243 (1999)	total: 6.14s	remaining: 55.2s
Stopped by overfitting detector  (100 

1000:	learn: 2617.7712666	test: 8525.7149542	best: 8525.7149542 (1000)	total: 1.73s	remaining: 32.8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 8356.877863
bestIteration = 1871

Shrink model to first 1872 iterations.
남은변수개수: 3
Learning rate set to 0.007527
0:	learn: 14436.1395640	test: 15057.8698301	best: 15057.8698301 (0)	total: 1.96ms	remaining: 39.2s
1000:	learn: 2798.7332181	test: 8139.9383509	best: 8139.9383509 (1000)	total: 1.86s	remaining: 35.2s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 7942.424953
bestIteration = 1470

Shrink model to first 1471 iterations.
남은변수개수: 2
Learning rate set to 0.007527
0:	learn: 14433.8844058	test: 15040.7537379	best: 15040.7537379 (0)	total: 1.83ms	remaining: 36.5s
1000:	learn: 5047.6717400	test: 6620.4853728	best: 6620.4853728 (1000)	total: 1.75s	remaining: 33.1s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 6524.647704
bestIteration = 1812

Shrink model to first 1813 iterations.

,변수명,FA단순가중,FA시간가중,FA경기가중,FA시간경기가중,NONFA단순가중,NONFA시간가중,NONFA경기가중,NONFA시간경기가중
0,포지션(수비)_유격수,145,142,144,145,137,142,141,144
1,팀명_NC,144,134,140,137,127,126,138,136
2,포지션(수비)_수비기록없음,143,143,145,144,140,138,143,142
3,포지션(수비)_3루수,142,138,141,135,136,131,135,130
4,팀명_우리/히어로즈/넥센/키움,141,145,142,143,113,118,123,116
...,...,...,...,...,...,...,...,...,...
140,5년평균타점,5,5,5,5,23,27,15,14
141,5년평균연봉,4,4,4,4,6,4,4,1
142,5년평균WAR,3,3,3,3,10,8,17,36
143,1차FA여부,2,2,2,2,49,48,56,53


In [37]:
cnt = 1

for pitcher_data, colname in zip(pitcher_datas, colnames):
    
    tmp = get_cb_RFE_rank(pitcher_data)
    tmp.columns = ['변수명', colname]
    
    if cnt == 1:
        pitcher_cb_rfe_score = tmp.copy()
    else:
        pitcher_cb_rfe_score = pd.merge(pitcher_cb_rfe_score, tmp, how='left', on='변수명')
        
    cnt += 1
    
pitcher_cb_rfe_score

남은변수개수: 143
Learning rate set to 0.006209
0:	learn: 48323.8358218	test: 39678.2798211	best: 39678.2798211 (0)	total: 9.16ms	remaining: 3m 3s
1000:	learn: 7919.6913904	test: 11857.9531478	best: 11857.9531478 (1000)	total: 10s	remaining: 3m 10s
2000:	learn: 3833.8608820	test: 10329.1614652	best: 10328.1945604 (1995)	total: 20.4s	remaining: 3m 3s
3000:	learn: 2237.5033353	test: 10028.1975061	best: 10027.7253126 (2999)	total: 30.2s	remaining: 2m 51s
4000:	learn: 1394.2744517	test: 9911.2477706	best: 9911.2477706 (4000)	total: 39.8s	remaining: 2m 39s
5000:	learn: 884.2154587	test: 9851.4210260	best: 9851.4210260 (5000)	total: 49.1s	remaining: 2m 27s
6000:	learn: 569.2429675	test: 9826.0463795	best: 9826.0183849 (5994)	total: 58.5s	remaining: 2m 16s
7000:	learn: 375.1246863	test: 9812.8071923	best: 9812.4012375 (6986)	total: 1m 8s	remaining: 2m 6s
8000:	learn: 252.6460424	test: 9805.7608297	best: 9805.7133998 (7997)	total: 1m 18s	remaining: 1m 57s
9000:	learn: 168.5443059	test: 9801.3187851	

1000:	learn: 7879.6825311	test: 12110.4587375	best: 12110.4587375 (1000)	total: 9.71s	remaining: 3m 4s
2000:	learn: 3975.4568578	test: 10557.6196849	best: 10557.6196849 (2000)	total: 19.1s	remaining: 2m 51s
3000:	learn: 2260.1594240	test: 10218.3363901	best: 10218.3363901 (3000)	total: 28.3s	remaining: 2m 40s
4000:	learn: 1412.1192522	test: 10125.3928208	best: 10125.2729831 (3999)	total: 37.5s	remaining: 2m 30s
5000:	learn: 889.9009593	test: 10084.0413893	best: 10084.0413893 (5000)	total: 47.1s	remaining: 2m 21s
6000:	learn: 575.3397244	test: 10055.4927538	best: 10055.4909457 (5983)	total: 56.9s	remaining: 2m 12s
7000:	learn: 379.2126472	test: 10040.0448673	best: 10040.0448673 (7000)	total: 1m 6s	remaining: 2m 4s
8000:	learn: 251.3820726	test: 10030.7122846	best: 10030.6763661 (7996)	total: 1m 16s	remaining: 1m 55s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 10028.95728
bestIteration = 8249

Shrink model to first 8250 iterations.
남은변수개수: 135
Learning rate set to 

2000:	learn: 3977.5957469	test: 10322.1283507	best: 10322.1283507 (2000)	total: 18.4s	remaining: 2m 45s
3000:	learn: 2276.4313735	test: 10087.7085793	best: 10087.7085793 (3000)	total: 27.4s	remaining: 2m 35s
4000:	learn: 1384.0667154	test: 10022.7808986	best: 10022.4980674 (3995)	total: 36.6s	remaining: 2m 26s
5000:	learn: 877.1561521	test: 9991.3827288	best: 9990.9228858 (4995)	total: 46.3s	remaining: 2m 18s
6000:	learn: 573.9746980	test: 9961.8150857	best: 9961.8150857 (6000)	total: 56s	remaining: 2m 10s
7000:	learn: 384.3024561	test: 9948.9271025	best: 9948.9271025 (7000)	total: 1m 5s	remaining: 2m 2s
8000:	learn: 255.2609480	test: 9943.6025328	best: 9943.5094040 (7989)	total: 1m 15s	remaining: 1m 53s
9000:	learn: 169.1602023	test: 9939.8842863	best: 9939.8339551 (8945)	total: 1m 25s	remaining: 1m 43s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 9939.614178
bestIteration = 9037

Shrink model to first 9038 iterations.
남은변수개수: 127
Learning rate set to 0.006209
0:

남은변수개수: 120
Learning rate set to 0.006209
0:	learn: 48346.8539050	test: 39679.8041948	best: 39679.8041948 (0)	total: 11.3ms	remaining: 3m 46s
1000:	learn: 7806.6858295	test: 11713.7190904	best: 11713.7190904 (1000)	total: 9.23s	remaining: 2m 55s
2000:	learn: 3831.5394295	test: 10197.3124125	best: 10197.3124125 (2000)	total: 18.4s	remaining: 2m 45s
3000:	learn: 2141.4927694	test: 9925.4953417	best: 9922.5749455 (2990)	total: 27.8s	remaining: 2m 37s
4000:	learn: 1336.1324856	test: 9861.2273936	best: 9861.2273936 (4000)	total: 37.3s	remaining: 2m 29s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 9861.042843
bestIteration = 4002

Shrink model to first 4003 iterations.
남은변수개수: 119
Learning rate set to 0.006209
0:	learn: 48354.0649696	test: 39683.6195799	best: 39683.6195799 (0)	total: 26.6ms	remaining: 8m 52s
1000:	learn: 7757.9536183	test: 11613.6015656	best: 11613.6015656 (1000)	total: 9.12s	remaining: 2m 53s
2000:	learn: 3745.0601882	test: 10152.6908280	best: 10152.69

4000:	learn: 1348.4575156	test: 9750.5061886	best: 9750.1219851 (3979)	total: 37.1s	remaining: 2m 28s
5000:	learn: 850.9289109	test: 9709.8451695	best: 9709.8451695 (5000)	total: 46.4s	remaining: 2m 19s
6000:	learn: 537.1651197	test: 9689.1233427	best: 9689.0693013 (5999)	total: 55.6s	remaining: 2m 9s
7000:	learn: 350.2796409	test: 9680.8157183	best: 9680.5438935 (6955)	total: 1m 4s	remaining: 1m 59s
8000:	learn: 232.2282174	test: 9677.0264448	best: 9677.0262121 (7999)	total: 1m 13s	remaining: 1m 50s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 9676.909845
bestIteration = 8010

Shrink model to first 8011 iterations.
남은변수개수: 111
Learning rate set to 0.006209
0:	learn: 48302.3211775	test: 39662.5306289	best: 39662.5306289 (0)	total: 11.7ms	remaining: 3m 54s
1000:	learn: 7676.4089976	test: 11435.3002902	best: 11435.3002902 (1000)	total: 9.28s	remaining: 2m 56s
2000:	learn: 3887.5904836	test: 9888.0296875	best: 9887.8432067 (1999)	total: 18.7s	remaining: 2m 48s
3000:	

...

2000:	learn: 1768.7491374	test: 9023.6047988	best: 9023.5484357 (1999)	total: 12.9s	remaining: 1m 56s
3000:	learn: 1172.8099719	test: 8960.5590626	best: 8960.2000633 (2994)	total: 19.6s	remaining: 1m 51s
4000:	learn: 824.2044904	test: 8935.0381717	best: 8934.5202156 (3941)	total: 25.9s	remaining: 1m 43s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 8934.520216
bestIteration = 3941

Shrink model to first 3942 iterations.
남은변수개수: 51
Learning rate set to 0.007464
0:	learn: 10666.5981942	test: 13486.4980014	best: 13486.4980014 (0)	total: 5.78ms	remaining: 1m 55s
1000:	learn: 2954.7560265	test: 9187.7483267	best: 9187.5444435 (999)	total: 6.06s	remaining: 1m 54s
2000:	learn: 1781.2597645	test: 8901.2324656	best: 8900.8728309 (1991)	total: 12.1s	remaining: 1m 48s
3000:	learn: 1195.7341020	test: 8839.5465984	best: 8839.3936730 (2998)	total: 18.3s	remaining: 1m 43s
4000:	learn: 840.5949724	test: 8812.5771914	best: 8812.3723952 (3992)	total: 24.3s	remaining: 1m 37s
Stopped 

1000:	learn: 2974.0685283	test: 9326.0769187	best: 9325.8124188 (999)	total: 4.99s	remaining: 1m 34s
2000:	learn: 1805.2990926	test: 9029.2092760	best: 9028.9044015 (1993)	total: 9.94s	remaining: 1m 29s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 8988.781057
bestIteration = 2478

Shrink model to first 2479 iterations.
남은변수개수: 37
Learning rate set to 0.007464
0:	learn: 10662.2301556	test: 13481.6878149	best: 13481.6878149 (0)	total: 4.69ms	remaining: 1m 33s
1000:	learn: 2966.2641751	test: 9388.3447359	best: 9388.3447359 (1000)	total: 4.91s	remaining: 1m 33s
2000:	learn: 1777.9088870	test: 9060.4738635	best: 9060.4738635 (2000)	total: 9.87s	remaining: 1m 28s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 9033.444349
bestIteration = 2637

Shrink model to first 2638 iterations.
남은변수개수: 36
Learning rate set to 0.007464
0:	learn: 10663.1321518	test: 13481.3672429	best: 13481.3672429 (0)	total: 5.16ms	remaining: 1m 43s
1000:	learn: 2978.6619899	test:

3000:	learn: 1234.3839166	test: 8863.7968087	best: 8863.7968087 (3000)	total: 10.1s	remaining: 57s
4000:	learn: 911.2636112	test: 8841.5800304	best: 8841.5800304 (4000)	total: 13.4s	remaining: 53.6s
5000:	learn: 699.9377012	test: 8825.9322325	best: 8825.9322325 (5000)	total: 16.9s	remaining: 50.6s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 8822.379489
bestIteration = 5467

Shrink model to first 5468 iterations.
남은변수개수: 24
Learning rate set to 0.007464
0:	learn: 10663.8279726	test: 13481.7724135	best: 13481.7724135 (0)	total: 3.34ms	remaining: 1m 6s
1000:	learn: 2964.9329897	test: 9140.7985249	best: 9140.7985249 (1000)	total: 3.36s	remaining: 1m 3s
2000:	learn: 1780.9237525	test: 8846.7611728	best: 8846.7611728 (2000)	total: 6.55s	remaining: 59s
3000:	learn: 1223.0134475	test: 8783.8154405	best: 8783.7502473 (2999)	total: 9.86s	remaining: 55.9s
4000:	learn: 908.9606112	test: 8760.6299853	best: 8760.6299853 (4000)	total: 13.1s	remaining: 52.5s
5000:	learn: 705.726

1000:	learn: 3001.6241613	test: 8833.7940248	best: 8833.7940248 (1000)	total: 2.96s	remaining: 56.3s
2000:	learn: 1857.6885754	test: 8572.9543475	best: 8572.9543475 (2000)	total: 5.87s	remaining: 52.8s
3000:	learn: 1361.9112879	test: 8502.4330830	best: 8502.4159672 (2997)	total: 8.75s	remaining: 49.6s
4000:	learn: 1064.5310343	test: 8455.1899515	best: 8455.1899515 (4000)	total: 11.7s	remaining: 46.8s
5000:	learn: 860.5809161	test: 8436.9450122	best: 8436.9450122 (5000)	total: 14.6s	remaining: 43.8s
6000:	learn: 718.9660240	test: 8425.1556599	best: 8424.9661808 (5983)	total: 17.5s	remaining: 40.7s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 8422.718413
bestIteration = 6384

Shrink model to first 6385 iterations.
남은변수개수: 14
Learning rate set to 0.007464
0:	learn: 10663.0296465	test: 13474.3231395	best: 13474.3231395 (0)	total: 3.23ms	remaining: 1m 4s
1000:	learn: 3032.5269097	test: 8714.1224880	best: 8714.0983707 (999)	total: 2.95s	remaining: 56s
2000:	learn: 1837.

,변수명,FA단순가중,FA시간가중,FA경기가중,FA시간경기가중,NONFA단순가중,NONFA시간가중,NONFA경기가중,NONFA시간경기가중
0,5년평균포지션 엔트로피,143,143,143,143,137,139,140,143
1,포지션 엔트로피,142,141,141,141,141,137,139,137
2,팀명_현대,141,139,139,138,124,124,126,126
3,팀명_KT,140,142,142,142,140,135,141,139
4,팀명_SK,139,140,140,140,84,82,112,86
...,...,...,...,...,...,...,...,...,...
138,5년평균WAR,5,5,7,7,2,2,6,6
139,데뷔년도,4,4,4,4,15,12,16,17
140,뉴스개수(누적정규화),3,3,2,2,3,4,5,5
141,5년평균뉴스개수(연도별정규화),2,2,3,3,8,11,11,13


#### (2) RFE 모델링

In [39]:
hitter_lgbm_rfe = pd.DataFrame()

for hitter_data, colname in zip(hitter_datas, colnames):
    
    # 성능을 담을 변수 초기화
    performances = []
    
    # train, test 분할
    X, y = Xy_split(hitter_data)
    
    # 변수개수별 성능파악
    for n_features in range(0, hitter_data.shape[1], 5):
        
        # 변수가 0개일때는 실행x
        if n_features == 0:
            continue

        ### 변수목록 선정
        features = list(hitter_lgbm_rfe_score.loc[hitter_lgbm_rfe_score[colname].isin([i for i in range(1, n_features+1)]), '변수명'])
        
        # 모델링 후 데이터프레임 생성
        X_tmp = X[features]
        performances.append(lgbm_model(X_tmp, y))
    
    tmp = pd.DataFrame({f'{colname}':performances}, index = [i for i in range(5, hitter_data.shape[1], 5)])
    hitter_lgbm_rfe = pd.concat([hitter_lgbm_rfe, tmp], axis=1)
    
hitter_lgbm_rfe



 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[49]	valid_0's rmse: 25051.6


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[48]	valid_0's rmse: 31339.6


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[21]	valid_0's rmse: 25792.1


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[274]	valid_0's rmse: 25972.8


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[98]	valid_0's rmse: 26513.7


 ============================ 1 ====================

Early stopping, best iteration is:
[373]	valid_0's rmse: 16649


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[109]	valid_0's rmse: 20484.8


 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[38]	valid_0's rmse: 19195.7


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[313]	valid_0's rmse: 22809


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[171]	valid_0's rmse: 14369.2


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[455]	valid_0's rmse

Early stopping, best iteration is:
[81]	valid_0's rmse: 14046.9


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[523]	valid_0's rmse: 16155.9


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[84]	valid_0's rmse: 21100.7


 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[36]	valid_0's rmse: 18268.3


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[96]	valid_0's rmse: 23259.8


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[94]	valid_0's rmse

Early stopping, best iteration is:
[49]	valid_0's rmse: 23159.5


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[94]	valid_0's rmse: 14402.4


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[365]	valid_0's rmse: 16799


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[58]	valid_0's rmse: 21583.9


 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[40]	valid_0's rmse: 19046.4


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[49]	valid_0's rmse: 

Early stopping, best iteration is:
[185]	valid_0's rmse: 24162.7


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[238]	valid_0's rmse: 27085.2


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[59]	valid_0's rmse: 21514.5


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[198]	valid_0's rmse: 21726.8


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's rmse: 25876.9
Early stopping, best iteration is:
[975]	valid_0's rmse: 25876.2


 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best

...

Early stopping, best iteration is:
[16]	valid_0's rmse: 7451.38


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[11]	valid_0's rmse: 3248.06


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's rmse: 16462.2
[2000]	valid_0's rmse: 16332.1
[3000]	valid_0's rmse: 16327.6
[4000]	valid_0's rmse: 16327.5
Early stopping, best iteration is:
[4292]	valid_0's rmse: 16327.5


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[9]	valid_0's rmse: 3744.65


 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[95]	valid_0's rmse: 10805.3


 ============================ 2 =====================

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[10]	valid_0's rmse: 3094.71


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's rmse: 13950.2
[2000]	valid_0's rmse: 13848.1
Early stopping, best iteration is:
[2163]	valid_0's rmse: 13846.9


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[10]	valid_0's rmse: 3781.5


 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's rmse: 9946.09
[2000]	valid_0's rmse: 9885
Early stopping, best iteration is:
[1991]	valid_0's rmse: 9884.87


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[26]	valid_0's rmse: 66

Early stopping, best iteration is:
[10]	valid_0's rmse: 3760.56


 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[116]	valid_0's rmse: 11081.2


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[16]	valid_0's rmse: 7645.01


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[11]	valid_0's rmse: 3122.97


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[53]	valid_0's rmse: 17710.9


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[10]	valid_0's rmse

Early stopping, best iteration is:
[144]	valid_0's rmse: 11250.5


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[18]	valid_0's rmse: 7726.55


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[12]	valid_0's rmse: 3107.51


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's rmse: 16338.2
[2000]	valid_0's rmse: 16175.3
[3000]	valid_0's rmse: 16168.4
[4000]	valid_0's rmse: 16168.2
Early stopping, best iteration is:
[4051]	valid_0's rmse: 16168.2


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[9]	valid_0's rmse: 3757.34


 ============================ 1 ====================

,FA단순가중,FA시간가중,FA경기가중,FA시간경기가중,NONFA단순가중,NONFA시간가중,NONFA경기가중,NONFA시간경기가중
5,26934.0,27440.4,27576.1,26377.9,7991.1,8190.1,8280.8,8571.5
10,26153.0,26229.0,26527.7,25686.3,8367.0,8650.7,7706.4,7982.5
15,24151.9,25634.2,25699.4,23987.0,8333.9,7503.5,7918.6,7818.9
20,17394.6,26098.3,24009.0,23814.7,7424.5,8265.9,7782.4,7738.5
25,17608.1,24153.4,18108.6,23174.8,7807.2,8235.0,7305.0,7769.4
30,18138.8,23887.4,18660.3,23554.9,8006.4,8203.1,7545.0,7431.5
35,18336.3,24073.1,18674.6,24038.4,8243.4,8159.2,7618.6,7506.3
40,18321.7,24640.9,18254.7,24287.0,8597.8,8092.4,8389.5,7775.1
45,18840.6,24562.7,18463.6,23883.5,8445.5,8247.2,8126.8,7797.9
50,18652.3,17556.1,18769.3,24265.7,8541.9,8168.0,8181.8,8388.3


In [40]:
pitcher_lgbm_rfe = pd.DataFrame()

for pitcher_data, colname in zip(pitcher_datas, colnames):
    
    # 성능을 담을 변수 초기화
    performances = []
    
    # train, test 분할
    X, y = Xy_split(pitcher_data)
    
    # 변수개수별 성능파악
    for n_features in range(0, pitcher_data.shape[1], 5):
        
        # 변수가 0개일때는 실행x
        if n_features == 0:
            continue

        ### 변수목록 선정
        features = list(pitcher_lgbm_rfe_score.loc[pitcher_lgbm_rfe_score[colname].isin([i for i in range(1, n_features+1)]), '변수명'])
        
        # 모델링 후 데이터프레임 생성
        X_tmp = X[features]
        performances.append(lgbm_model(X_tmp, y))
    
    tmp = pd.DataFrame({f'{colname}':performances}, index = [i for i in range(5, pitcher_data.shape[1], 5)])
    pitcher_lgbm_rfe = pd.concat([pitcher_lgbm_rfe, tmp], axis=1)
    
pitcher_lgbm_rfe



 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[115]	valid_0's rmse: 17881.9


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[115]	valid_0's rmse: 27114.9


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[161]	valid_0's rmse: 26105.2


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[66]	valid_0's rmse: 25901.2


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[135]	valid_0's rmse: 24079.9


 ============================ 1 =================

Early stopping, best iteration is:
[57]	valid_0's rmse: 23844.6


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[142]	valid_0's rmse: 22048.6


 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[76]	valid_0's rmse: 20287.7


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[380]	valid_0's rmse: 24587.1


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[136]	valid_0's rmse: 25994.5


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[56]	valid_0's rm

Early stopping, best iteration is:
[406]	valid_0's rmse: 16936.7


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[228]	valid_0's rmse: 18698.8


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's rmse: 12687.7
Early stopping, best iteration is:
[903]	valid_0's rmse: 12687.4


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[75]	valid_0's rmse: 12962.4


 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[318]	valid_0's rmse: 10817.6


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best

Early stopping, best iteration is:
[62]	valid_0's rmse: 13082.3


 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[361]	valid_0's rmse: 11389.6


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[364]	valid_0's rmse: 17242.8


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[188]	valid_0's rmse: 18290.6


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[754]	valid_0's rmse: 12680.4


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[62]	valid_0's r

[177]	valid_0's rmse: 24727.8


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[139]	valid_0's rmse: 20019.7


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[224]	valid_0's rmse: 26309.1


 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[156]	valid_0's rmse: 17219.1


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[324]	valid_0's rmse: 22994.9


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[127]	valid_0's rmse: 24873.1


 =================

...

Early stopping, best iteration is:
[16]	valid_0's rmse: 3773.3


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[107]	valid_0's rmse: 3794.74


 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[330]	valid_0's rmse: 4630.56


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[193]	valid_0's rmse: 12422


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[20]	valid_0's rmse: 7350.08


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[16]	valid_0's rmse:

[12000]	valid_0's rmse: 12141.1
[13000]	valid_0's rmse: 12141.1
[14000]	valid_0's rmse: 12141.1
[15000]	valid_0's rmse: 12141.1
[16000]	valid_0's rmse: 12141.1
Early stopping, best iteration is:
[16049]	valid_0's rmse: 12141.1


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[19]	valid_0's rmse: 7635.4


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[23]	valid_0's rmse: 3854.79


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[62]	valid_0's rmse: 4116.09


 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[23]	valid_0's rmse: 5095.59


 ===============

Early stopping, best iteration is:
[22]	valid_0's rmse: 3757.66


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[76]	valid_0's rmse: 3798.71


 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[582]	valid_0's rmse: 4714.3


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[235]	valid_0's rmse: 12286.4


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[21]	valid_0's rmse: 7533.16


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[20]	valid_0's rmse

Early stopping, best iteration is:
[27]	valid_0's rmse: 7528.99


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[22]	valid_0's rmse: 3729.92


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[94]	valid_0's rmse: 3757.97


 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[818]	valid_0's rmse: 4709.04


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[255]	valid_0's rmse: 12497.8


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[27]	valid_0's rms

,FA단순가중,FA시간가중,FA경기가중,FA시간경기가중,NONFA단순가중,NONFA시간가중,NONFA경기가중,NONFA시간경기가중
5,24216.6,23668.8,24127.8,25758.3,6856.9,6873.4,6715.9,6802.8
10,22435.5,23163.4,22214.2,23562.0,5874.2,6308.3,6346.8,6434.1
15,22724.9,21332.5,22851.8,23472.0,6105.7,6516.2,6464.4,6552.2
20,23183.7,22210.6,22345.5,22579.0,6261.1,6404.8,6243.8,6469.4
25,23393.6,21556.0,22341.4,22882.6,6253.9,6321.6,6389.6,6385.6
30,23109.8,21504.8,22234.0,22868.5,6299.9,6349.2,6401.1,6449.1
35,23247.3,21978.3,22404.5,22814.4,6324.6,6381.7,6398.7,6371.6
40,23132.2,21959.7,22152.2,22905.1,6383.3,6350.4,6418.1,6416.5
45,23327.8,19759.8,23009.0,23728.5,6355.3,6376.7,6333.7,6454.3
50,23002.0,19976.1,23077.9,23555.1,6295.9,6369.2,6360.8,6461.8


In [42]:
hitter_cb_rfe = pd.DataFrame()

for hitter_data, colname in zip(hitter_datas, colnames):
    
    # 성능을 담을 변수 초기화
    performances = []
    
    # train, test 분할
    X, y = Xy_split(hitter_data)
    
    # 변수개수별 성능파악
    for n_features in range(0, hitter_data.shape[1], 5):
        
        # 변수가 0개일때는 실행x
        if n_features == 0:
            continue

        ### 변수목록 선정
        features = list(hitter_cb_rfe_score.loc[hitter_cb_rfe_score[colname].isin([i for i in range(1, n_features+1)]), '변수명'])
        
        # 모델링 후 데이터프레임 생성
        X_tmp = X[features]
        performances.append(cb_model(X_tmp, y))
    
    tmp = pd.DataFrame({f'{colname}':performances}, index = [i for i in range(5, hitter_data.shape[1], 5)])
    hitter_cb_rfe = pd.concat([hitter_cb_rfe, tmp], axis=1)
    
hitter_cb_rfe

Learning rate set to 0.007132
0:	learn: 50861.5681865	test: 49291.8348508	best: 49291.8348508 (0)	total: 2.14ms	remaining: 42.8s
1000:	learn: 14116.4169112	test: 16667.4928196	best: 16658.3440769 (981)	total: 1.18s	remaining: 22.4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 16636.08326
bestIteration = 1078

Shrink model to first 1079 iterations.
Learning rate set to 0.007132
0:	learn: 49420.1697193	test: 54859.7684234	best: 54859.7684234 (0)	total: 1.3ms	remaining: 26s
1000:	learn: 13220.4495329	test: 24529.9928240	best: 24529.9928240 (1000)	total: 1.24s	remaining: 23.6s
2000:	learn: 10941.8704865	test: 24118.5296355	best: 24117.0632520 (1997)	total: 2.48s	remaining: 22.4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 24114.78575
bestIteration = 2011

Shrink model to first 2012 iterations.
Learning rate set to 0.007132
0:	learn: 51093.9198256	test: 48301.9561998	best: 48301.9561998 (0)	total: 2.64ms	remaining: 52.8s
1000:	learn: 13524.402944

KeyboardInterrupt: 

In [ ]:
pitcher_cb_rfe = pd.DataFrame()

for pitcher_data, colname in zip(pitcher_datas, colnames):
    
    # 성능을 담을 변수 초기화
    performances = []
    
    # train, test 분할
    X, y = Xy_split(pitcher_data)
    
    # 변수개수별 성능파악
    for n_features in range(0, pitcher_data.shape[1], 5):
        
        # 변수가 0개일때는 실행x
        if n_features == 0:
            continue

        ### 변수목록 선정
        features = list(pitcher_cb_rfe_score.loc[pitcher_cb_rfe_score[colname].isin([i for i in range(1, n_features+1)]), '변수명'])
        
        # 모델링 후 데이터프레임 생성
        X_tmp = X[features]
        performances.append(cb_model(X_tmp, y))
    
    tmp = pd.DataFrame({f'{colname}':performances}, index = [i for i in range(5, pitcher_data.shape[1], 5)])
    pitcher_cb_rfe = pd.concat([pitcher_cb_rfe, tmp], axis=1)
    
pitcher_cb_rfe

# 6. Feature selection (Mutual Information)

#### (1) 순위도출

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression

In [ ]:
cnt = 1

for hitter_data, colname in zip(hitter_datas, colnames):

    # train, test 분할
    X, y = Xy_split(hitter_data)

    # score 산정
    mutual_info = SelectKBest(mutual_info_regression, k=1).fit(X, y)
    mutual_info_importance = pd.Series(mutual_info.scores_).rank(ascending=False)

    # 데이터프레임 생성
    tmp = pd.DataFrame({'변수명':list(X.columns),
                        f'{colname}':list(mutual_info_importance)})
    
    if cnt == 1:
        hitter_mi_score = tmp.copy()
    else:
        hitter_mi_score = pd.merge(hitter_mi_score, tmp, how='left', on='변수명')
        
    cnt += 1
    
hitter_mi_score

In [ ]:
cnt = 1

for pitcher_data, colname in zip(pitcher_datas, colnames):

    # train, test 분할
    X, y = Xy_split(pitcher_data)

    # score 산정
    mutual_info = SelectKBest(mutual_info_regression, k=1).fit(X, y)
    mutual_info_importance = pd.Series(mutual_info.scores_).rank(ascending=False)

    # 데이터프레임 생성
    tmp = pd.DataFrame({'변수명':list(X.columns),
                        f'{colname}':list(mutual_info_importance)})
    
    if cnt == 1:
        pitcher_mi_score = tmp.copy()
    else:
        pitcher_mi_score = pd.merge(pitcher_mi_score, tmp, how='left', on='변수명')
        
    cnt += 1
    
pitcher_mi_score

#### (2) 성능도출

In [ ]:
hitter_lgbm_mi = pd.DataFrame()

for hitter_data, colname in zip(hitter_datas, colnames):
    
    # 성능을 담을 변수 초기화
    performances = []
    
    # train, test 분할
    X, y = Xy_split(hitter_data)
    
    # 변수개수별 성능파악
    for n_features in range(0, hitter_data.shape[1], 5):
        
        # 변수가 0개일때는 실행x
        if n_features == 0:
            continue

        ### 변수목록 선정
        features = list(hitter_mi_score.loc[hitter_mi_score[colname].isin([i for i in range(1, n_features+1)]), '변수명'])
        
        # 모델링 후 데이터프레임 생성
        X_tmp = X[features]
        performances.append(lgbm_model(X_tmp, y))
    
    tmp = pd.DataFrame({f'{colname}':performances}, index = [i for i in range(5, hitter_data.shape[1], 5)])
    hitter_lgbm_mi = pd.concat([hitter_lgbm_mi, tmp], axis=1)
    
hitter_lgbm_mi

In [ ]:
hitter_cb_mi = pd.DataFrame()

for hitter_data, colname in zip(hitter_datas, colnames):
    
    # 성능을 담을 변수 초기화
    performances = []
    
    # train, test 분할
    X, y = Xy_split(hitter_data)
    
    # 변수개수별 성능파악
    for n_features in range(0, hitter_data.shape[1], 5):
        
        # 변수가 0개일때는 실행x
        if n_features == 0:
            continue

        ### 변수목록 선정
        features = list(hitter_mi_score.loc[hitter_mi_score[colname].isin([i for i in range(1, n_features+1)]), '변수명'])
        
        # 모델링 후 데이터프레임 생성
        X_tmp = X[features]
        performances.append(cb_model(X_tmp, y))
    
    tmp = pd.DataFrame({f'{colname}':performances}, index = [i for i in range(5, hitter_data.shape[1], 5)])
    hitter_cb_mi = pd.concat([hitter_cb_mi, tmp], axis=1)
    
hitter_cb_mi

In [ ]:
pitcher_lgbm_mi = pd.DataFrame()

for pitcher_data, colname in zip(pitcher_datas, colnames):
    
    # 성능을 담을 변수 초기화
    performances = []
    
    # train, test 분할
    X, y = Xy_split(pitcher_data)
    
    # 변수개수별 성능파악
    for n_features in range(0, pitcher_data.shape[1], 5):
        
        # 변수가 0개일때는 실행x
        if n_features == 0:
            continue

        ### 변수목록 선정
        features = list(pitcher_mi_score.loc[pitcher_mi_score[colname].isin([i for i in range(1, n_features+1)]), '변수명'])
        
        # 모델링 후 데이터프레임 생성
        X_tmp = X[features]
        performances.append(lgbm_model(X_tmp, y))
    
    tmp = pd.DataFrame({f'{colname}':performances}, index = [i for i in range(5, pitcher_data.shape[1], 5)])
    pitcher_lgbm_mi = pd.concat([pitcher_lgbm_mi, tmp], axis=1)
    
pitcher_lgbm_mi

In [ ]:
pitcher_cb_mi = pd.DataFrame()

for pitcher_data, colname in zip(pitcher_datas, colnames):
    
    # 성능을 담을 변수 초기화
    performances = []
    
    # train, test 분할
    X, y = Xy_split(pitcher_data)
    
    # 변수개수별 성능파악
    for n_features in range(0, pitcher_data.shape[1], 5):
        
        # 변수가 0개일때는 실행x
        if n_features == 0:
            continue

        ### 변수목록 선정
        features = list(pitcher_mi_score.loc[pitcher_mi_score[colname].isin([i for i in range(1, n_features+1)]), '변수명'])
        
        # 모델링 후 데이터프레임 생성
        X_tmp = X[features]
        performances.append(cb_model(X_tmp, y))
    
    tmp = pd.DataFrame({f'{colname}':performances}, index = [i for i in range(5, pitcher_data.shape[1], 5)])
    pitcher_cb_mi = pd.concat([pitcher_cb_mi, tmp], axis=1)
    
pitcher_cb_mi